In [1]:
import pandas as pd
import numpy as np
import re

from elder_care.config import SRC

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
def table(df_col):
    return pd.crosstab(df_col, columns="Count")["Count"]

In [4]:
def process_gv_imputations(wave, args):
    module = "gv_imputations"
    module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
    data = pd.read_stata(module_file, convert_categoricals=False)

    # Filter the data based on the "country" column
    data = data[data["country"] == 12]

    # # Select columns 'mergeid' and the specified args
    # selected_columns = ["mergeid"] + args
    # data = data[selected_columns]

    # Select columns 'mergeid' and the specified args (create missing columns with NaN)
    # Filter existing columns
    selected_columns = ["mergeid"] + [col for col in args if col in data.columns]
    columns = ["mergeid"] + args

    # Create missing columns and fill with NaN
    for col in args:
        if col not in selected_columns:
            data[col] = np.nan

    # data = data[selected_columns]
    data = data[columns]

    # Replace negative values with NaN using NumPy
    # data[args] = np.where(data[args] >= 0, data[args], np.nan)

    # Group the data by 'mergeid'
    grouped_data = data.groupby("mergeid")

    # Create a dictionary to store the aggregation method for each column
    aggregation_methods = {}
    for column in args:
        dtype = data[column].dtype
        if pd.api.types.is_integer_dtype(dtype):
            aggregation_methods[column] = "median"
        elif pd.api.types.is_float_dtype(dtype):
            aggregation_methods[column] = "mean"

    # Replace negative values with NaN using NumPy
    # this should not change the meaning except for cases where all 5 entries are missing
    # check ?!
    data[args] = np.where(data[args] >= 0, data[args], np.nan)
    # data[args] = np.where(data[args] >= 0, data[args], 0)

    # Apply aggregation methods and store the results in a new DataFrame
    aggregated_data = grouped_data.agg(aggregation_methods).reset_index()

    # if "age_p" in args:
    #    # note that single people also have partner_alive = 0
    #    aggregated_data['partner_alive'] = np.where(aggregated_data['age_p'] > 0, 1, 0)

    aggregated_data["wave"] = wave

    return aggregated_data

In [5]:
# Sample data
data = {
    "ID": [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    "Values": [1, 2, np.nan, 1, np.nan, np.nan, 1, 3, 5, np.nan],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Group by 'ID'
grouped_data = df.groupby("ID")

# Define aggregation methods
aggregation_methods = {
    "Values": "median",
}

# Apply aggregation methods and store the results in a new DataFrame
aggregated_data = grouped_data.agg(aggregation_methods).reset_index()

aggregated_data

,ID,Values
0,1,1.0
1,2,3.0


In [6]:
wave = 2
module = "gv_imputations"
module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
data = pd.read_stata(module_file, convert_categoricals=False)
data = data[data["country"] == 12]

In [7]:
args = [
    "gender",
    "age",  # Age of respondent (based on interview year)
    "age_p",  # Age of partner (based on interview year)
    "mstat",  # Marital status
    "single",
    "couple",
    "partner",
    "nursinghome",
    # "perho",
    "ydip",
    "yind",
    "ypen1",  # Annual old age, early retirement pensions, survivor and war pension
    "ypen2",  # Annual private occupational pensions
    "ypen5",  # Annual payment from social assistance
    "yreg1",  # Other regular payments from private pensions
    "yreg2",  # Other regular payment from private transfer
    "thinc",  # Total household net income - version A
    "thinc2",  # Total household net income - version B
    "hnetw",  # Household net worth (hnfass + hrass)
    "yedu",
    "yedu_p",
    "isced",
    "sphus",  # Self-perceived health – US scale
    "nchild",  # Number of children
    "gali",  # Limitation with activities: PH005
    "chronic",  # Number of chronic diseases: PH006
    "adl",  # Limitations with activities of daily living: PH049_1
    "iadl",  # Limitations with instrumental activities of daily living: PH049_2
    "eurod",  # EURO depression scale: MH002-MH017 (MH031)
    "cjs",  # Current job situation: EP005
    "pwork",  # Did any paid work: EP002
    "empstat",  # Employee or self-employed: EP009; 2- 8
    "rhfo",  # Received help from others (how many): SP002, SP005, SP007
    "ghto",  # Given help to others (how many): SP008, SP011, SP013
    "ghih",  # Given help in the household (how many): SP0181 2 4 5 6 7 (R) 8
    "rhih",  # Received help in the household (how many): SP0201 2 4 5 6 7 (R) 8
    "otrf",  # Owner, tenant or rent free: HO0021 2 4 5 6 7 (R) 8
]
# args = ["mstat"]

# thinc: (sum of ydip, ypen1, ypen2, ypen3, ypen4, ypen5,
# ypen6, yreg1, yreg2, yind, ysrent, yaohm, ybabsmf)

# hrass: Household real assets
# (home*perho/100+vbus*sbus/100+car+ ores – mort)
# hgfass: Household gross financial assets (sum of
# bacc, bsmf and slti)
# hnfass: Household net financial assets (hgfass – liab)

# Select columns 'mergeid' and the specified args (create missing columns with NaN)
# Filter existing columns
selected_columns = ["mergeid"] + [col for col in args if col in data.columns]
columns = ["mergeid"] + args

# Create missing columns and fill with NaN
for col in args:
    if col not in selected_columns:
        data[col] = np.nan

# data = data[selected_columns]
data = data[columns]


# Group the data by 'mergeid'
grouped_data = data.groupby("mergeid")

In [8]:
result_df = process_gv_imputations(8, args)
# result_df["single"].isna().sum()
result_df["couple"].isna().sum()

table(result_df["ghto"])

ghto
0.0    1853
1.0     701
2.0     240
3.0      84
Name: Count, dtype: int64

In [9]:
table(result_df["ghih"])

ghih
0.0    2064
1.0     196
2.0       3
Name: Count, dtype: int64

In [10]:
table(data["age_p"])

age_p
-99    2590
 25       5
 33       5
 35       5
 36       5
 37       5
 38      10
 39      15
 40       5
 41      10
 42      10
 43      45
 44      40
 45      25
 46      20
 47      30
 48      65
 49      95
 50     120
 51     225
 52     265
 53     330
 54     395
 55     406
 56     455
 57     460
 58     431
 59     377
 60     305
 61     376
 62     260
 63     465
 64     440
 65     305
 66     515
 67     460
 68     355
 69     455
 70     385
 71     382
 72     325
 73     242
 74     160
 75     145
 76     160
 77     155
 78     160
 79     125
 80     106
 81     120
 82      90
 83      65
 84      55
 85      30
 86      30
 87       5
 88      10
 89       5
Name: Count, dtype: int64

In [11]:
(data["age_p"] < 40).sum() / 5, data.shape[0] / 5

(528.0, 2628.0)

In [12]:
def test_negative_age_couple_condition(data):
    # Check that if 'age_p' is negative or missing, 'couple' is 0
    condition = ((data["age_p"] < 0) | data["age_p"].isna()) == (data["couple"] == 0)
    assert condition.all()

In [13]:
test_negative_age_couple_condition(data)

In [14]:
data[
    (data["age_p"] < 0)
    | (data["age_p"].isna()) & (data["couple"] == 0) & (data["single"] == 1)
].shape

(2590, 37)

In [15]:
data[(data["age_p"] < 0) | (data["age_p"].isna()) & (data["couple"] == 0)].shape

(2590, 37)

In [16]:
data[(data["age_p"] < 0) | (data["age_p"].isna()) & (data["single"] == 0)].shape

(2590, 37)

In [17]:
aggregation_methods = {}
for column in args:
    dtype = data[column].dtype
    if pd.api.types.is_integer_dtype(dtype):
        aggregation_methods[column] = "median"
    elif pd.api.types.is_float_dtype(dtype):
        aggregation_methods[column] = "mean"

In [18]:
aggregation_methods

{'gender': 'median',
 'age': 'median',
 'age_p': 'median',
 'mstat': 'median',
 'single': 'median',
 'couple': 'median',
 'partner': 'median',
 'nursinghome': 'median',
 'ydip': 'mean',
 'yind': 'mean',
 'ypen1': 'mean',
 'ypen2': 'mean',
 'ypen5': 'mean',
 'yreg1': 'mean',
 'yreg2': 'mean',
 'thinc': 'mean',
 'thinc2': 'mean',
 'hnetw': 'mean',
 'yedu': 'mean',
 'yedu_p': 'mean',
 'isced': 'median',
 'sphus': 'median',
 'nchild': 'median',
 'gali': 'median',
 'chronic': 'median',
 'adl': 'median',
 'iadl': 'median',
 'eurod': 'median',
 'cjs': 'median',
 'pwork': 'median',
 'empstat': 'median',
 'rhfo': 'median',
 'ghto': 'median',
 'ghih': 'median',
 'rhih': 'median',
 'otrf': 'median'}

In [19]:
aggregated_data = grouped_data.agg(aggregation_methods).reset_index()
aggregated_data

,mergeid,gender,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf
0,DE-000132-01,2.0,53.0,-99.0,5.0,1.0,0.0,-99.0,0.0,20000.000000,0.0,0.0,0.0,0.0,0.0,0.0,21850.000000,22896.000000,113310.178834,17.0,-99.0,5.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,-99.0,1.0,2.0,1.0,0.0,0.0,1.0
1,DE-001381-01,2.0,53.0,61.0,5.0,0.0,1.0,1.0,0.0,1200.000000,0.0,0.0,0.0,0.0,0.0,0.0,8995.740918,12000.000000,32779.784972,13.0,11.0,5.0,5.0,2.0,0.0,1.0,0.0,0.0,5.0,3.0,1.0,1.0,3.0,2.0,0.0,-99.0,3.0
2,DE-001381-02,1.0,61.0,53.0,5.0,0.0,1.0,2.0,0.0,900.000000,0.0,0.0,0.0,0.0,0.0,0.0,8995.740918,12000.000000,32779.784972,11.0,13.0,3.0,5.0,2.0,1.0,5.0,0.0,0.0,6.0,3.0,0.0,-99.0,3.0,1.0,0.0,0.0,3.0
3,DE-001748-01,1.0,50.0,43.0,1.0,0.0,1.0,1.0,0.0,18591.740766,0.0,0.0,0.0,0.0,0.0,0.0,22254.731445,62846.383338,39821.574948,9.0,13.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,-99.0,1.0,0.0,1.0,0.0,0.0,3.0
4,DE-001748-02,2.0,43.0,50.0,1.0,0.0,1.0,2.0,0.0,1100.000000,0.0,0.0,0.0,0.0,0.0,0.0,22254.731445,62846.383338,39821.574948,13.0,9.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,-99.0,1.0,0.0,0.0,0.0,-99.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,DE-998490-02,2.0,50.0,59.0,2.0,0.0,1.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,15900.000000,14400.000000,10000.000000,12.0,12.0,3.0,4.0,2.0,1.0,1.0,0.0,0.0,6.0,3.0,0.0,-99.0,3.0,1.0,0.0,0.0,4.0
2624,DE-999375-01,1.0,70.0,68.0,1.0,0.0,1.0,1.0,0.0,0.000000,0.0,15600.0,1200.0,0.0,0.0,0.0,21690.134375,21600.000000,138869.948142,10.0,18.0,3.0,5.0,3.0,1.0,6.0,0.0,1.0,6.0,1.0,0.0,-99.0,0.0,1.0,0.0,-99.0,1.0
2625,DE-999375-02,2.0,68.0,70.0,1.0,0.0,1.0,2.0,0.0,0.000000,0.0,4800.0,0.0,0.0,0.0,0.0,21690.134375,21600.000000,138869.948142,18.0,10.0,3.0,4.0,3.0,1.0,7.0,3.0,2.0,6.0,5.0,0.0,-99.0,0.0,0.0,0.0,0.0,1.0
2626,DE-999404-01,1.0,60.0,58.0,4.0,0.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4800.000000,14400.000000,69058.099347,10.0,12.0,3.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,-99.0,0.0,0.0,0.0,0.0,1.0


In [20]:
(aggregated_data["single"] == "couple").sum()

0

In [21]:
((aggregated_data["single"] == 0) & (aggregated_data["couple"] == 0)).sum()

0

In [22]:
result_df = process_gv_imputations(8, args)
# result_df["single"].isna().sum()
result_df["couple"].isna().sum()

0

In [23]:
pd.crosstab(data["mstat"], data["couple"])

couple,0,1
mstat,,
1,60,9905
2,5,61
3,95,46
4,454,166
5,577,202
6,1399,170


In [24]:
result_df["partner_alive"] = np.where(result_df["age_p"] > 0, 1, 0)

In [25]:
# 1, 2, 3
result_df[
    ((result_df["mstat"] == 1) | (result_df["mstat"] == 2) | (result_df["mstat"] == 3))
    & ((result_df["age_p"] < 0) | (result_df["age_p"].isna()))
]

,mergeid,gender,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,wave,partner_alive
67,DE-022865-01,2.0,72.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,27600.000000,0.0,0.000000,0.0,0.0,28100.000000,27600.0,412000.000000,25.0,NaN,5.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,NaN,1.0,0.0,NaN,NaN,1.0,8,0
113,DE-043035-01,2.0,65.0,NaN,3.0,1.0,0.0,NaN,0.0,16000.0,0.0,0.000000,0.0,0.000000,0.0,0.0,16188.263867,14400.0,93169.164179,15.0,NaN,4.0,5.0,2.0,1.0,5.0,0.0,0.0,6.0,2.0,NaN,1.0,2.0,1.0,NaN,NaN,3.0,8,0
132,DE-051099-01,1.0,76.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,11400.000000,0.0,0.000000,0.0,0.0,11600.000000,17400.0,3689.752249,15.0,NaN,5.0,3.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,1.0,NaN,NaN,3.0,8,0
220,DE-081577-01,1.0,75.0,NaN,3.0,1.0,0.0,NaN,0.0,0.0,0.0,14400.000000,0.0,0.000000,0.0,0.0,14400.000000,14400.0,3513.691221,11.0,NaN,3.0,4.0,3.0,1.0,3.0,0.0,0.0,3.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,3.0,8,0
248,DE-092185-01,2.0,77.0,NaN,2.0,1.0,0.0,NaN,0.0,0.0,0.0,15600.000000,0.0,0.000000,0.0,0.0,15610.000000,15600.0,163000.000000,11.0,NaN,3.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,DE-947557-02,2.0,83.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,7440.000000,0.0,624.820659,0.0,0.0,31285.452734,30000.0,16874.458822,12.0,NaN,3.0,5.0,0.0,1.0,1.0,4.0,5.0,6.0,1.0,0.0,NaN,2.0,0.0,0.0,0.0,3.0,8,0
2757,DE-953610-01,2.0,77.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,18000.000000,0.0,0.000000,0.0,0.0,18000.000000,24000.0,250200.000000,10.2,NaN,5.0,4.0,2.0,1.0,3.0,0.0,4.0,2.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
2841,DE-987303-01,2.0,81.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,18414.087666,0.0,0.000000,0.0,0.0,22043.488477,24000.0,159957.684939,14.0,NaN,3.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
2846,DE-987811-01,1.0,75.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,14400.000000,400.0,0.000000,0.0,0.0,22064.807031,42000.0,710707.598509,11.0,NaN,3.0,4.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,NaN,0.0,0.0,1.0,NaN,1.0,8,0


In [26]:
cols = ["mstat"]
result_df = process_gv_imputations(2, cols)

assert result_df["mergeid"].nunique() == len(result_df)
assert result_df["mstat"].count() == len(result_df)

result_df["mstat"].count(), result_df["mstat"].isna().sum()

(2628, 0)

In [27]:
wave = 2
module = "dn"
module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
dn = pd.read_stata(module_file, convert_categoricals=False)
dn = dn[dn["country"] == 12]

dn["dn014_"].count(), dn["dn014_"].isna().sum(), dn["dn014_"].shape

(1102, 1526, (2628,))

In [28]:
def load_and_rename_wave_data(wave):
    module = "sp"
    module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"

    data = pd.read_stata(module_file, convert_categoricals=False)
    data.columns = [col[:-2] if col.endswith("sp") else col for col in data.columns]

    return data

In [29]:
data = load_and_rename_wave_data(wave=4)
data[data["sp019d1"].notna()]["sp019d1"]

31       1.0
34       1.0
112      0.0
138      1.0
168      1.0
        ... 
57940    0.0
57957    0.0
57960    0.0
57965    0.0
57973    1.0
Name: sp019d1, Length: 3905, dtype: float64

In [30]:
def filter_nested_dict(original_dict, keys_to_remove):
    return {
        key: [value for value in values if value not in keys_to_remove.get(key, [])]
        if key in keys_to_remove
        else values
        for key, values in original_dict.items()
    }

In [31]:
def process_wave(wave_number, data_modules):
    wave_data = {}

    for module in data_modules.keys():
        print(f"module: {module}")
        module_file = (
            SRC / f"data/sharew{wave_number}/sharew{wave_number}_rel8-0-0_{module}.dta"
        )

        # Read and filter
        if module in ["re", "rp"] and wave_number == 7:
            wave_module = pd.read_stata(module_file, convert_categoricals=False)
            wave_module = wave_module[wave_module["country"] == 12]

            lookup = {
                f"{var[3:]}": f"{var}"
                for var in data_modules[module]
                if var.startswith("sl")
            }

        else:
            wave_module = pd.read_stata(module_file, convert_categoricals=False)
            # wave_module = wave_module[wave_module["country"] == "Germany"]
            wave_module = wave_module[wave_module["country"] == 12]

            lookup = {
                "sp009_1sp": "sp009_1",
                "sp009_2sp": "sp009_2",
                "sp009_3sp": "sp009_3",
                "sp019d1sp": "sp019d1",
                "sp019d2sp": "sp019d2",
                "sp019d3sp": "sp019d3",
                "sp019d4sp": "sp019d4",
                "sp019d5sp": "sp019d5",
                "sp019d6sp": "sp019d6",
                "sp019d7sp": "sp019d7",
                "sp019d8sp": "sp019d8",
                "sp019d9sp": "sp019d9",
                "sp019d10sp": "sp019d10",
                "sp019d11sp": "sp019d11",
                "sp019d12sp": "sp019d12",
                "sp019d13sp": "sp019d13",
                "sp019d14sp": "sp019d14",
                "sp019d15sp": "sp019d15",
                "sp019d16sp": "sp019d16",
                "sp019d17sp": "sp019d17",
                "sp019d18sp": "sp019d18",
                "sp019d19sp": "sp019d19",
                "sp019d20sp": "sp019d20",
                "sp019d21sp": "sp019d21",
                # received personal care within household
                "sp021d1sp": "sp021d1",
                "sp021d2sp": "sp021d2",
                "sp021d3sp": "sp021d3",
                "sp021d4sp": "sp021d4",
                "sp021d5sp": "sp021d5",
                "sp021d6sp": "sp021d6",
                "sp021d7sp": "sp021d7",
                "sp021d8sp": "sp021d8",
                "sp021d9sp": "sp021d9",
                "sp021d10sp": "sp021d10",
                "sp021d11sp": "sp021d11",
                "sp021d12sp": "sp021d12",
                "sp021d13sp": "sp021d13",
                "sp021d14sp": "sp021d14",
                "sp021d15sp": "sp021d15",
                "sp021d16sp": "sp021d16",
                "sp021d17sp": "sp021d17",
                "sp021d18sp": "sp021d18",
                "sp021d19sp": "sp021d19",
                "sp021d20sp": "sp021d20",
                "sp021d21sp": "sp021d21",
            }
            # result_dict = {f"sp021d{num}sp": f"sp021d{num}" for num in range(1, 22)}

        # Rename columns using the dictionary
        wave_module.rename(columns=lookup, inplace=True)

        module_vars = ["mergeid"] + data_modules[module]

        # Select columns
        wave_module = wave_module[module_vars]

        wave_data[module] = wave_module

        print(wave_module.shape)

    add_wealth_data = "gv_imputations" in data_modules
    merged_data = wave_data["cv_r"]

    data_modules.pop("cv_r")
    data_modules.pop("gv_imputations", None)

    for module_key in data_modules.keys():
        merged_data = merged_data.merge(
            wave_data[module_key],
            on="mergeid",
            how="outer",
        )

    if add_wealth_data:
        merged_data = merged_data.merge(
            wave_data["gv_imputations"],
            on="mergeid",
            how="left",
        )

    merged_data["wave"] = wave_number

    return merged_data

In [32]:
all_variables = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "dn": [
        "dn002_",
        "dn003_",
        "dn019_",  # widowed since when
        "dn010_",
        "dn041_",
        "dn009_",
        "dn014_",
        "dn015_",
        "dn016_",
        "dn026_1",
        "dn026_2",
        "dn033_1",
        "dn033_2",
        "dn027_1",
        "dn027_2",
        "dn028_1",
        "dn028_2",
        "dn030_1",
        "dn030_2",
        "dn034_",  # any siblings
        "dn036_",  # how many brothers alive
        "dn037_",  # how many sisters alive
        "dn127_1",
        "dn127_2",
        "dn032_1",
        "dn032_2",
        "dn012d1",
        "dn012d2",
        "dn012d3",
        "dn012d4",
        "dn012d5",
        "dn012d6",
        "dn012d7",
        "dn012d8",
        "dn012d9",
        "dn012d10",
        "dn012d11",
        "dn012d12",
        "dn012d13",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
        # "dn012d95",
        "dn012dno",
        "dn012dot",
    ],
    "ep": [
        "ep005_",
        "ep013_",
        "ep002_",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
        "ep328_",
        "ep329_",
        "ep213_1",
        # "ep213_2",
        # "ep213_3",
        # "ep213_4",
        # "ep213_5",
        # "ep213_6",
        # "ep213_7",
        # "ep213_8",
        # "ep213_9",
        # "ep213_10",
        # "ep213_11",
        # "ep213_12",
        # "ep213_13",
        # "ep213_14",
        # "ep213_15",
        # "ep213_16",
    ],
    "sp": [
        # outside household
        "sp008_",  # given help outside
        "sp009_1",  # to whom given help outside 1
        "sp009_2",  # to whom given help outside 2
        "sp009_3",  # to whom given help outside 3
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
        # only wave 1 and 2
        # sp012_1, # number of hours practical help
        #
        "sp011_1",  # how often given help to person 1
        "sp011_2",  # how often given help to person 1
        "sp011_3",  # how often given help to person 1
        "sp013_1",  # GiveHelpToOth
        "sp013_2",  # GiveHelpToOth
        # "sp013_3",  # GiveHelpToOth
        # within household
        "sp018_",  # given help within
        "sp019d1",  # provided help with personal care to: spouse/partner
        "sp019d2",  # provided help with personal care to: mother
        "sp019d3",  # provided help with personal care to: father
        "sp019d4",
        "sp019d5",
        "sp019d6",
        "sp019d7",
        "sp019d8",
        "sp019d9",
        "sp019d10",
        "sp019d11",
        # "sp019d12",
        # "sp019d13",
        # "sp019d14",
        # "sp019d15",
        # "sp019d16",
        # "sp019d17",
        # "sp019d18",
        # "sp019d19",
        # "sp019d20",
        # received personal care in household
        "sp020_",  # someone in this household helped you regularly with personal care
        "sp021d1",  # R received help with personal care from: spouse/partner
        "sp021d10",  # child 1
        "sp021d11",  # child 2
        "sp021d12",  # child 3
        "sp021d13",  # child 4
        "sp021d14",  # child 5
        "sp021d15",  # child 6
        "sp021d16",  # child 7
        "sp021d17",  # child 8
        "sp021d18",  # child 9
        "sp021d19",  # child other
        "sp021d20",  # son in law
        "sp021d21",  # daughter in law
    ],
    "gv_isced": ["isced1997_r"],
    # "gv_imputations": [
    #    "hnetw"
    # ],  # household net worth = total gross financial assets + total real assets - total libailities
    "ch": ["ch001_"],
}

In [33]:
keys_to_remove_wave1 = {
    "dn": [
        "dn041_",
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "ep": [
        "ep013_",
        "ep328_",
        "ep329_",
        "ep213_12",
        "ep213_13",
        "ep213_14",
        "ep213_15",
        "ep213_16",
    ],
}

keys_to_remove_wave2 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
}


keys_to_remove_wave4 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "sp": [
        # type of help not answered, assume help includes personal care
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
        # within household personal care to brother etc.
        # "sp019d8",
        # "sp019d9",
        "sp019d10",
        "sp019d11",
        # provided help with personal care to child 3 - 9
        "sp019d12",
        "sp019d13",
        "sp019d14",
        "sp019d15",
        "sp019d16",
        "sp019d17",
        "sp019d18",
        "sp019d19",
        "sp019d20",
        #
        # received help with personal care from child 3 - 9
        "sp021d10",
        "sp021d11",
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
        "sp021d20",
        "sp021d21",
    ],
}


keys_to_remove_wave5 = {
    "dn": [
        "dn127_1",  # year of death mother
        "dn127_2",  # year of death father
        "dn012d20",  # further educ category 20
        "dn012dno",  # further educ none
    ],
    # type of help not answered, assume help includes personal care
    "sp": [
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
    ],
}

keys_to_remove_wave6 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

keys_to_remove_wave7 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    "re": [
        "sl_re018_1",
        "sl_re018_2",
        "sl_re018_3",
        "sl_re018_4",
        "sl_re018_5",
        "sl_re018_6",
        "sl_re018_7",
        "sl_re018_8",
        "sl_re018_9",
        "sl_re018_10",
        "sl_re018_11",
        "sl_re018_12",
        "sl_re018_13",
        "sl_re018_14",
        "sl_re018_15",
        "sl_re018_16",
        "sl_re018_17",
        "sl_re020_1",
        "sl_re020_2",
        "sl_re020_3",
        "sl_re020_4",
        "sl_re020_5",
        "sl_re020_6",
        "sl_re020_7",
        "sl_re020_8",
        "sl_re020_9",
        "sl_re020_10",
        "sl_re020_11",
        "sl_re020_12",
        "sl_re020_13",
        "sl_re020_14",
        "sl_re020_15",
        "sl_re020_16",
        "sl_re020_17",
    ],
    "rp": [
        "sl_rp003_18",
        "sl_rp008_6",
    ],  # year started living with partner, year married
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

keys_to_remove_wave8 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

In [34]:
# Retrospective waves
re_vars = (
    [f"sl_re011_{i}" for i in range(1, 21)]
    + [f"sl_re016_{i}" for i in range(1, 21)]
    + [f"sl_re026_{i}" for i in range(1, 21)]
    + [f"sl_re018_{i}" for i in range(1, 17)]
    + [f"sl_re020_{i}" for i in range(1, 17)]
)
rp_vars = (
    ["sl_rp002_", "sl_rp002d_", "sl_rp002e_"]
    + [f"sl_rp003_{i}" for i in range(11, 19)]  # year started living with partner
    + [
        f"sl_rp004b_{i}" for i in range(1, 6)
    ]  # year started living with married partner
    # year started living with partner
    + [f"sl_rp004c_{i}" for i in [1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 16, 17]]
    + [f"sl_rp008_{i}" for i in range(1, 7)]  # year married
    + [f"sl_rp013_{i}" for i in range(1, 5)]  #  divorced partner
    + [f"sl_rp014_{i}" for i in range(1, 5)]  # year divorced partner
)

rp_vars_wave3 = [
    f"sl_rp011_{i}" for i in [1, 2, 3, 11, 12, 13, 14, 15]
]  # 16 year death of partner
rp_vars_wave7 = [f"sl_rp011_{i}" for i in [1, 2, 3, 4, 11, 12, 13, 14, 15]]

# Data modules for wave 3
variables_wave3 = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "re": re_vars,
    "rp": rp_vars + rp_vars_wave3,
}

# Separate modules for partly retrospective wave 7
variables_wave7 = filter_nested_dict(
    all_variables | {"re": re_vars, "rp": rp_vars + rp_vars_wave7},
    keys_to_remove_wave7,
)

In [35]:
variables_wave1 = filter_nested_dict(all_variables, keys_to_remove_wave1)
variables_wave2 = filter_nested_dict(all_variables, keys_to_remove_wave2)
variables_wave4 = filter_nested_dict(all_variables, keys_to_remove_wave4)
variables_wave5 = filter_nested_dict(all_variables, keys_to_remove_wave5)
variables_wave6 = filter_nested_dict(all_variables, keys_to_remove_wave6)
variables_wave8 = filter_nested_dict(all_variables, keys_to_remove_wave8)

In [36]:
wave1 = process_wave(wave_number=1, data_modules=variables_wave1)
wave2 = process_wave(wave_number=2, data_modules=variables_wave2)
wave3 = process_wave(wave_number=3, data_modules=variables_wave3)
wave4 = process_wave(wave_number=4, data_modules=variables_wave4)
wave5 = process_wave(wave_number=5, data_modules=variables_wave5)
wave6 = process_wave(wave_number=6, data_modules=variables_wave6)
wave7 = process_wave(wave_number=7, data_modules=variables_wave7)
wave8 = process_wave(wave_number=8, data_modules=variables_wave8)

module: cv_r
(3920, 8)
module: dn
(2995, 40)
module: ep
(2995, 14)
module: sp
(2995, 39)
module: gv_isced
(2995, 2)
module: ch
(2995, 2)
module: cv_r
(3504, 8)
module: dn
(2628, 41)
module: ep
(2628, 17)
module: sp
(2628, 39)
module: gv_isced
(2628, 2)
module: ch
(2628, 2)
module: cv_r
(2501, 8)
module: re
(1918, 93)
module: rp
(1918, 51)
module: cv_r
(2146, 8)
module: dn
(1619, 40)
module: ep
(1619, 17)
module: sp
(1619, 22)
module: gv_isced
(1619, 2)
module: ch
(1619, 2)
module: cv_r
(7674, 8)
module: dn
(5750, 45)
module: ep
(5750, 17)
module: sp
(5750, 36)
module: gv_isced
(5750, 2)
module: ch
(5750, 2)
module: cv_r
(5787, 8)
module: dn
(4411, 48)
module: ep
(4411, 7)
module: sp
(4411, 31)
module: gv_isced
(4411, 2)
module: ch
(4411, 2)
module: cv_r
(4928, 8)
module: dn
(3820, 48)
module: ep
(3820, 7)
module: sp
(3820, 31)
module: gv_isced
(3820, 2)
module: ch
(3820, 2)
module: re
(3820, 61)
module: rp
(3820, 50)
module: cv_r
(4202, 8)
module: dn
(2878, 48)
module: ep
(2878, 7)
mod

In [37]:
# merge in gv imputations by wave
gv_vars = [
    "gender",
    "age",  # Age of respondent (based on interview year)
    "age_p",  # Age of partner (based on interview year)
    "mstat",  # Marital status
    "single",
    "couple",
    "partner",
    "nursinghome",
    # "perho",
    "ydip",
    "yind",
    "ypen1",  # Annual old age, early retirement pensions, survivor and war pension
    "ypen2",  # Annual private occupational pensions
    "ypen5",  # Annual payment from social assistance
    "yreg1",  # Other regular payments from private pensions
    "yreg2",  # Other regular payment from private transfer
    "thinc",  # Total household net income - version A
    "thinc2",  # Total household net income - version B
    "hnetw",  # Household net worth (hnfass + hrass)
    "yedu",
    "yedu_p",
    "isced",
    "sphus",  # Self-perceived health – US scale
    "nchild",  # Number of children
    "gali",  # Limitation with activities: PH005
    "chronic",  # Number of chronic diseases: PH006
    "adl",  # Limitations with activities of daily living: PH049_1
    "iadl",  # Limitations with instrumental activities of daily living: PH049_2
    "eurod",  # EURO depression scale: MH002-MH017 (MH031)
    "cjs",  # Current job situation: EP005
    "pwork",  # Did any paid work: EP002
    "empstat",  # Employee or self-employed: EP009; 2- 8
    "rhfo",  # Received help from others (how many): SP002, SP005, SP007
    "ghto",  # Given help to others (how many): SP008, SP011, SP013
    "ghih",  # Given help in the household (how many): SP0181 2 4 5 6 7 (R) 8
    "rhih",  # Received help in the household (how many): SP0201 2 4 5 6 7 (R) 8
    "otrf",  # Owner, tenant or rent free: HO0021 2 4 5 6 7 (R) 8
]

In [38]:
gv_wave1 = process_gv_imputations(wave=1, args=gv_vars)
gv_wave2 = process_gv_imputations(wave=2, args=gv_vars)
gv_wave4 = process_gv_imputations(wave=4, args=gv_vars)
gv_wave5 = process_gv_imputations(wave=5, args=gv_vars)
gv_wave6 = process_gv_imputations(wave=6, args=gv_vars)
gv_wave7 = process_gv_imputations(wave=7, args=gv_vars)
gv_wave8 = process_gv_imputations(wave=8, args=gv_vars)

wave1_merged = wave1.merge(gv_wave1, on='mergeid', how='left')
wave2_merged = wave1.merge(gv_wave2, on='mergeid', how='left')

wave4_merged = wave1.merge(gv_wave4, on='mergeid', how='left')
wave5_merged = wave1.merge(gv_wave5, on='mergeid', how='left')
wave6_merged = wave1.merge(gv_wave6, on='mergeid', how='left')
wave7_merged = wave1.merge(gv_wave7, on='mergeid', how='left')
wave8_merged = wave1.merge(gv_wave8, on='mergeid', how='left')

In [39]:
# List of dataset variable names
dataset_names = ["wave1", "wave2", "wave3", "wave4", "wave5", "wave6", "wave7", "wave8"]

# Create a list to store the counts of duplicated mergeid values for each dataset
duplicated_counts = []

for dataset_name in dataset_names:
    # Get the DataFrame for the current dataset
    df = globals()[dataset_name]  # Access the DataFrame using the variable name

    # Check for duplicates in the 'mergeid' column
    duplicates = df["mergeid"].duplicated().sum()

    # Append the count to the list
    duplicated_counts.append((dataset_name, duplicates))

# Print the counts for each dataset
for dataset_name, duplicates in duplicated_counts:
    print(f"{dataset_name}: {duplicates} duplicated mergeid values.")

wave1: 0 duplicated mergeid values.
wave2: 0 duplicated mergeid values.
wave3: 0 duplicated mergeid values.
wave4: 0 duplicated mergeid values.
wave5: 0 duplicated mergeid values.
wave6: 0 duplicated mergeid values.
wave7: 0 duplicated mergeid values.
wave8: 0 duplicated mergeid values.


In [40]:
waves_list = [wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8]
# waves_list = [wave1, wave2, wave3, wave4, wave5, wave6, wave7]

# Drop all nan rows
for i, df in enumerate(waves_list):
    waves_list[i] = df.dropna(how="all", axis=0, inplace=False)
    # waves_list[i] = df.dropna(axis=1, how='all')

In [41]:
def merge_wave_datasets(wave_datasets):
    # Combine the data frames in wave_datasets into one data frame
    #     combined_data = pd.concat(wave_datasets)
    combined_data = pd.concat(wave_datasets, axis=0, ignore_index=True)

    # Filter out rows where the 'int_year' column is not equal to -9
    # combined_data = combined_data[combined_data["int_year"] != -9]
    # combined_data = combined_data[combined_data["int_year"] != "Not applicable"]
    combined_data = combined_data[combined_data["int_year"] != -9]

    # Sort the data frame by 'mergeid' and 'int_year'
    combined_data = combined_data.sort_values(by=["mergeid", "int_year"])

    return combined_data

In [42]:
data = merge_wave_datasets(waves_list)

In [43]:
data.shape

# (26593, 251)

(26593, 256)

In [44]:
gv_wave_list = [gv_wave1, gv_wave2, gv_wave4, gv_wave5, gv_wave6, gv_wave7, gv_wave8]

# Concatenate the DataFrames vertically
stacked_gv_data = pd.concat(gv_wave_list, axis=0, ignore_index=True)

# Sort the DataFrame by 'mergeid' and 'wave'
stacked_gv_data = stacked_gv_data.sort_values(by=["mergeid", "wave"])

# Reset the index after sorting
stacked_gv_data = stacked_gv_data.reset_index(drop=True)

In [45]:
stacked_gv_data.drop("gender", axis=1, inplace=True)

# Merge 'data' and 'stacked_gv_data' on 'mergeid' and 'wave' with a left join
data = data.merge(stacked_gv_data, on=["mergeid", "wave"], how="left")

In [46]:
data.shape

# (26593, 251)

(26593, 291)

In [47]:
sum(pd.crosstab(data["int_year"], columns="Count")["Count"])

26590

In [48]:
def table(df_col):
    return pd.crosstab(df_col, columns="Count")["Count"]

# 2) Data preparation

## Discrete state variables

###### states which are interpolated
- labor market experience (0, 30)
- years in retirement (0, 6)
- years in intensive care (0, 5)
- father age (70, 90)
- mother age (70, 90)
###### states which are not interpolated
- individuals’ type (1, 2)
- father died last period (0, 1)
- mother died last period (0, 1)
- father alive (0, 1)
- mother alive (0, 1)
- health of father (1, 2, 3)
- health of mother (1, 2, 3)
- existence of siblings (0, 1) --> **change to existence of sister?**
- parents live close by (0, 1)
- married (0, 1)
- education (low, high)

In [49]:
MIN_AGE, MAX_AGE = 55, 68

In [50]:
# only females
dat = data.copy()

# Filter for females
# dat = dat[dat['gender'] == "Female"]
dat = dat[dat["gender"] == 2]

In [51]:
dat.shape

(14153, 291)

In [52]:
dat["dn036_"].isna().sum(), dat["dn037_"].isna().sum()

(3805, 3765)

In [53]:
dat["dn036_"].notna().sum(), dat["dn037_"].notna().sum()

(10348, 10388)

In [54]:
table(dat["dn036_"])

dn036_
-2.0        4
-1.0       28
 0.0     3671
 1.0     4270
 2.0     1589
 3.0      539
 4.0      163
 5.0       59
 6.0       17
 7.0        3
 8.0        1
 9.0        1
 11.0       2
 20.0       1
Name: Count, dtype: int64

In [55]:
# number of siblings alive
# dat["siblings"] = dat["dn036_"] + dat["dn037_"]

# Set negative values to missing
dat["dn036_"] = np.where(dat["dn036_"] < 0, np.nan, dat["dn036_"])
dat["dn037_"] = np.where(dat["dn037_"] < 0, np.nan, dat["dn037_"])

dat["siblings"] = np.select(
    [
        (~dat["dn036_"].isna()) & (~dat["dn037_"].isna()),  # Both columns are not NaN
        (~dat["dn036_"].isna()) & dat["dn037_"].isna(),  # Only dn036_ is not NaN
        dat["dn036_"].isna() & (~dat["dn037_"].isna()),  # Only dn037_ is not NaN
        (dat["dn036_"].isna()) & (dat["dn037_"].isna()),  # Both columns are NaN
    ],
    [
        dat["dn036_"] + dat["dn037_"],  # Addition when both columns are not NaN
        dat["dn036_"],  # Value from dn036_ when only dn036_ is not NaN
        dat["dn037_"],  # Value from dn037_ when only dn037_ is not NaN
        np.nan,  # Result is NaN when both columns are NaN
    ],
    default=np.nan,
)

In [56]:
dat["siblings"].notna().sum()

10729

In [57]:
table(dat["siblings"])

siblings
0.0      994
1.0     3820
2.0     2929
3.0     1450
4.0      770
5.0      359
6.0      202
7.0      103
8.0       58
9.0       27
10.0       3
11.0       5
12.0       2
13.0       3
14.0       3
20.0       1
Name: Count, dtype: int64

In [58]:
# Age calculation
dat["age"] = dat.apply(
    lambda row: row["int_year"] - row["yrbirth"]
    if row["int_month"] >= row["mobirth"]
    else row["int_year"] - row["yrbirth"] - 1,
    axis=1,
)

# Keep only those aged 55 to 68
dat = dat[(dat["age"] >= MIN_AGE) & (dat["age"] <= MAX_AGE)]

# Rename 'dn041_' to 'years_educ'
dat.rename(columns={"dn041_": "years_educ"}, inplace=True)

In [59]:
conditions = [
    (dat["years_educ"] < 0),
    (dat["years_educ"] > 25),
]
values = [np.nan, np.nan]

# Use numpy.select to set values in the 'years_educ' column based on conditions
dat["years_educ"] = np.select(conditions, values, dat["years_educ"])

# Create 'high_educ' column, setting NaN when 'years_educ' is NaN
dat["high_educ"] = np.where(
    dat["years_educ"].isna(),
    np.nan,
    (dat["years_educ"] >= 15).astype(int),
)

In [60]:
# Create a list of further education columns
further_educ = [
    "dn012d1",
    "dn012d2",
    "dn012d3",
    "dn012d4",
    "dn012d5",
    "dn012d6",
    "dn012d7",
    "dn012d8",
    "dn012d9",
    "dn012d10",
    "dn012d11",
    "dn012d12",
    "dn012d13",
    "dn012d14",
    "dn012d15",
    "dn012d16",
    "dn012d17",
    "dn012d18",
    "dn012d19",
    "dn012d20",
    #'dn012d95' # currently in education --> not needed
]

In [61]:
for educ in further_educ:
    number = int(re.search(r"\d+", educ).group())
    conditions = [
        dat[educ] < 0,
        dat[educ] == number,
    ]
    values = [np.nan, 1]

    dat[educ] = np.select(conditions, values, dat[educ])

In [62]:
dat["dn012dno"] = np.where(dat["dn012dno"] < 0, np.nan, dat["dn012dno"])
dat["dn012dot"] = np.where(dat["dn012dot"] < 0, np.nan, dat["dn012dot"])
dat["dn012dno"] = np.where(dat["dn012dno"] == 1, 0, dat["dn012dno"])

def find_max_suffix(row):
    max_suffix = 0
    for col in further_educ:
        if row[col] == 1:
            suffix = int(col.split('dn012d')[-1])
            max_suffix = max(max_suffix, suffix)
            
    return max_suffix if max_suffix >= 0 else np.nan

dat['further_educ_max'] = dat.apply(find_max_suffix, axis=1)

In [63]:
def find_max_suffix(row):
    active_cols = [
        int(col.split("dn012d")[-1]) for col in further_educ if row[col] == 1
    ]
    max_suffix = max(active_cols) if active_cols else 0
    return max_suffix if max_suffix >= 0 else np.nan


dat["further_educ_max"] = dat.apply(find_max_suffix, axis=1)

In [64]:
dat["high_educ_012"] = (
    (dat["wave"].isin([1, 2, 4]) & (dat["further_educ_max"] >= 3))
    | (dat["wave"].between(5, 7) & (dat["further_educ_max"] >= 10))
).astype(int)

# dat.loc[dat["further_educ_max"].isna(), "high_educ_012"] = np.nan

dat["high_educ_comb"] = ((dat["high_educ"] == 1) | (dat["high_educ_012"] == 1)).astype(
    int,
)

In [65]:
table(dat["high_educ_comb"])

high_educ_comb
0    6238
1     585
Name: Count, dtype: int64

In [66]:
dat = dat.rename(columns={"ch001_": "nchild"})
dat["nchild"] = np.where(dat["nchild"] >= 0, dat["nchild"], np.nan)

In [67]:
# Current job situation
# -2 Refusal
# -1 Don't know
# 1 Retired
# 2 Employed or self-employed (including working for family business)
# 3 Unemployed
# 4 Permanently sick or disabled
# 5 Homemaker
# 97 Other


dat["ep005_"] = np.where(dat["ep005_"] >= 0, dat["ep005_"], np.nan)

In [68]:
conditions = [
    (dat["ep005_"] == 1) | ~dat["ep329_"].isna(),
    dat["ep005_"].isna() & dat["ep329_"].isna(),
]

values = [1, np.nan]

dat["retired"] = np.select(conditions, values, 0)

In [69]:
# retirement year and month
dat["ep329_"] = np.where(dat["ep329_"] >= 0, dat["ep329_"], np.nan)  # year
dat["ep328_"] = np.where(dat["ep328_"] >= 0, dat["ep328_"], np.nan)  # month

# more nans in month 5473 > 5088
# use lagged retirement!!!

In [70]:
conditions = [
    (dat["retired"] == 1) | ~dat["ep329_"].isna(),
    (dat["retired"] == 0),
]

values = [dat["int_year"] - dat["ep329_"], np.nan]

dat["years_since_retirement"] = np.select(conditions, values, np.nan)

dat["_years_since_retirement"] = dat["int_year"] - dat["ep329_"]

In [71]:
table(dat["retired"])

retired
0.0    3750
1.0    2443
Name: Count, dtype: int64

In [72]:
table(dat["years_since_retirement"])

years_since_retirement
0.0     153
1.0     282
2.0     282
3.0     236
4.0     192
5.0     119
6.0     113
7.0      87
8.0      84
9.0      50
10.0     27
11.0     17
12.0     16
13.0     12
14.0     14
15.0     12
16.0      6
17.0      7
18.0      6
20.0      4
21.0      2
22.0      2
23.0      2
24.0      3
26.0      2
34.0      1
45.0      1
46.0      2
47.0      1
Name: Count, dtype: int64

In [73]:
table(
    dat["ypen1"],
)  # Annual old age, early retirement pensions, survivor and war pension

ypen1
0.0        4023
60.0          1
200.0         1
250.0         1
500.0         2
           ... 
38400.0       2
40800.0       1
45600.0       1
49200.0       1
54000.0       1
Name: Count, Length: 872, dtype: int64

In [74]:
(dat["ypen1"] > 0).sum()

2208

In [75]:
conditions = [
    (dat["ep005_"] == 1)
    | (~dat["ep329_"].isna() & (dat["ep329_"] <= dat["int_year"]))
    | (dat["ypen1"] > 0),  # or >= 200??
    dat["ep005_"].isna() & dat["ep329_"].isna(),
]

values = [1, np.nan]

dat["retired"] = np.select(conditions, values, 0)

In [76]:
table(dat["retired"])

# 0.0    3750
# 1.0    2443

retired
0.0    3430
1.0    2782
Name: Count, dtype: int64

In [77]:
dat["retired"].mean()

0.4478428847392144

In [78]:
dat["years_since_retirement"].mean()

4.255907780979827

In [79]:
dat["years_since_retirement"].notna().sum()

1735

In [80]:
dat["_years_since_retirement"].notna().sum()

1735

# ToDo

If the information is missing and individuals are considered retired we use information given in SHARElife
to construct retrospectively the year in which the last job ended.

In [81]:
table(dat["ep329_"])

ep329_
1960.0      1
1967.0      1
1968.0      1
1969.0      1
1979.0      1
1981.0      1
1987.0      1
1989.0      5
1990.0      2
1991.0      3
1992.0      4
1993.0      9
1995.0      7
1996.0      9
1997.0      7
1998.0     23
1999.0     30
2000.0     35
2001.0     22
2002.0     39
2003.0     70
2004.0     65
2005.0     84
2006.0     85
2007.0     73
2008.0     85
2009.0    130
2010.0    106
2011.0    183
2012.0    157
2013.0     73
2014.0     90
2015.0     91
2016.0     71
2017.0     60
2018.0     49
2019.0     60
2020.0      1
Name: Count, dtype: int64

In [82]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Group the DataFrame by 'mergeid' and fill missing values within each group
dat["dn019_"] = dat.groupby("mergeid")["dn019_"].transform(lambda x: x.ffill().bfill())

## Married

In [83]:
# Partner We use marriage information in SHARE to construct an indicator on the existence of
# a partner living in the same household. We do not distinguish between marriage and registered
# partnership.

conditions_married_or_partner = [
    dat["mstat"].isin([1, 2]),
    dat["mstat"].isin([3, 4, 5, 6]),
]
values_married_or_partner = [1, 0]
# replace with zeros or nans
dat["married"] = np.select(
    conditions_married_or_partner,
    values_married_or_partner,
    np.nan,
)

In [84]:
table(dat["married"])

married
0.0    1481
1.0    4750
Name: Count, dtype: int64

In [85]:
dat["married"].mean()

0.7623174450329

In [86]:
4847 / (4847 + 1372)

0.7793857533365492

In [87]:
conditions_married_or_partner = [
    dat["dn014_"].isin([1, 2]),
    dat["dn014_"].isin([3, 4, 5, 6]),
]
values_married_or_partner = [1, 0]
# replace with zeros or nans
dat["married_dn"] = np.select(
    conditions_married_or_partner,
    values_married_or_partner,
    np.nan,
)

In [88]:
table(dat["married_dn"])

married_dn
0.0     561
1.0    1743
Name: Count, dtype: int64

# Caregiving

In [89]:
table(dat["ghto"])

ghto
0.0    2835
1.0    1360
2.0     419
3.0     160
Name: Count, dtype: int64

In [90]:
table(dat["ghih"])

ghih
0.0    4201
1.0     323
2.0      13
Name: Count, dtype: int64

In [91]:
table(dat["sp008_"])

sp008_
-2.0       1
-1.0       1
 1.0    1936
 5.0    2829
Name: Count, dtype: int64

In [92]:
table(dat["sp018_"])

sp018_
1.0     335
5.0    4179
Name: Count, dtype: int64

In [93]:
table(dat["sp011_1"])  # how often

sp011_1
-2.0      1
-1.0      1
 1.0    412
 2.0    641
 3.0    382
 4.0    499
Name: Count, dtype: int64

In [94]:
# Update 'sp008_' to handle negative values
dat["sp008_"] = dat["sp008_"].apply(lambda x: x if x >= 0 else np.nan)
# dat['sp008_'] = np.where(dat['sp008_'] < 0, np.nan, dat['sp008_'])

In [95]:
# to whom given help within household, available for all waves:)

# Update 'sp009_1', 'sp009_2', and 'sp009_3' to handle negative values
columns_to_update = ["sp009_1", "sp009_2", "sp009_3"]
for col in columns_to_update:
    dat[col] = dat[col].apply(lambda x: x if x >= 0 else np.nan)

In [96]:
table(dat["sp008_"])

sp008_
1.0    1936
5.0    2829
Name: Count, dtype: int64

In [97]:
table(dat["sp018_"])

sp018_
1.0     335
5.0    4179
Name: Count, dtype: int64

In [98]:
# Assuming 'dat' is a pandas DataFrame

# Create the 'ever_cared' column
dat["ever_cared"] = np.where(
    (dat["sp008_"] == 1) | (dat["sp018_"] == 1),
    1,
    np.where(
        ((dat["sp008_"] == 5) & (dat["sp018_"] == 5))
        | ((dat["sp008_"] == 5) & dat["sp018_"].isna())
        | (dat["sp008_"].isna() & (dat["sp018_"] == 5)),
        0,
        # 0,
        np.nan,
    ),
)

In [99]:
table(dat["ever_cared"])

# 0.0    3193
# 1.0    2157

ever_cared
0.0    3195
1.0    2157
Name: Count, dtype: int64

In [100]:
table(dat["ever_cared"])

# 0.0    3193
# 1.0    2157

ever_cared
0.0    3195
1.0    2157
Name: Count, dtype: int64

In [101]:
dat["ever_cared"].notna().sum()

5352

In [102]:
# Create the 'ever_cared' column
conditions_ever_cared = [
    (dat["sp008_"] == 1) | (dat["sp018_"] == 1),
    ((dat["sp008_"] == 5) & (dat["sp018_"] == 5))
    | ((dat["sp008_"] == 5) & dat["sp018_"].isna())
    | (dat["sp008_"].isna() & (dat["sp018_"] == 5)),
]

choices_ever_cared = [1, 0]

dat["ever_cared"] = np.select(conditions_ever_cared, choices_ever_cared, default=np.nan)

In [103]:
table(dat["ever_cared"])

ever_cared
0.0    3195
1.0    2157
Name: Count, dtype: int64

In [104]:
# "sp010d1_1",  # help given person 1: personal care missing in waves 4, 5
# "sp019d1" - 09,  # provided help with personal care to: spouse/partner etc in ALL waves
# "sp019d2",  # provided help with personal care to: mother
# "sp019d3",  # provided help with personal care to: father


# Create the 'ever_cared_parents_outside' column
conditions_parents_outside = [
    (dat["sp008_"] == 1)
    & (
        (dat["sp009_1"].isin([2, 3]))  # to whom
        | (dat["sp009_2"].isin([2, 3]))  # to whom
        | (dat["sp009_3"].isin([2, 3]))  # to whom
    ),
    dat["sp008_"].isna(),
]

choices_parents_outside = [1, np.nan]

dat["ever_cared_parents_outside"] = np.select(
    conditions_parents_outside,
    choices_parents_outside,
    default=0,
)

In [105]:
table(dat["ever_cared_parents_outside"])

ever_cared_parents_outside
0.0    4100
1.0     665
Name: Count, dtype: int64

In [106]:
# Create the 'ever_cared_parents_within' column
conditions_parents_within = [
    (dat["sp018_"] == 1) & ((dat["sp019d2"] == 1) | (dat["sp019d3"] == 1)),
    dat["sp018_"].isna(),
]

choices_parents_within = [1, np.nan]

dat["ever_cared_parents_within"] = np.select(
    conditions_parents_within,
    choices_parents_within,
    default=0,
)

# Create the 'ever_cared_parents' column
conditions_parents = [
    (dat["ever_cared_parents_outside"] == 1) | (dat["ever_cared_parents_within"] == 1),
    (dat["ever_cared_parents_within"].isna())
    & (dat["ever_cared_parents_outside"].isna()),
]

choices_parents = [1, np.nan]

dat["ever_cared_parents"] = np.select(conditions_parents, choices_parents, default=0)

In [107]:
table(dat["ever_cared_parents_within"])

ever_cared_parents_within
0.0    4442
1.0      72
Name: Count, dtype: int64

In [108]:
table(dat["ever_cared_parents_outside"])

ever_cared_parents_outside
0.0    4100
1.0     665
Name: Count, dtype: int64

In [109]:
table(dat["ever_cared_parents"])

ever_cared_parents
0.0    4624
1.0     728
Name: Count, dtype: int64

In [110]:
(
    table(dat["ever_cared_parents_within"]).sum(),
    table(dat["ever_cared_parents_outside"]).sum(),
    table(dat["ever_cared_parents"]).sum(),
)

(4514, 4765, 5352)

In [111]:
# Define conditions and choices for np.select
conditions = [
    (dat["sp018_"] == 1) & ((dat["sp019d2"] == 1) | (dat["sp019d3"] == 1)),
    (dat["sp008_"] == 1)
    & ((dat["sp009_1"] == 2) | (dat["sp009_2"] == 2) | (dat["sp009_3"] == 2)),
    (dat["sp008_"] == 1)
    & ((dat["sp009_1"] == 3) | (dat["sp009_2"] == 3) | (dat["sp009_3"] == 3)),
]

choices = [1, 1, 1]  # Assign 1 if the conditions are met

# Use np.select to create the 'care_in_year' column
dat["care_in_year"] = np.select(conditions, choices, default=0)
# dat.loc[
#    ((dat["sp008_"]).isna() & ((dat["sp019d2"] == 1) | (dat["sp019d3"] == 1))),
#    "care_in_year",
# ] = np.nan

In [112]:
table(dat["sp008_"])

sp008_
1.0    1936
5.0    2829
Name: Count, dtype: int64

In [113]:
table(dat["care_in_year"])

care_in_year
0    6095
1     728
Name: Count, dtype: int64

In [114]:
table(dat["ever_cared_parents"])

ever_cared_parents
0.0    4624
1.0     728
Name: Count, dtype: int64

In [115]:
# not finished!
# add: experience in informal care
# add periods in which person gave informal care (no matter whether outside or within household)
# experience + 1 in next period! (do not count this years informal care as experience)

In [116]:
dat = dat.sort_values(by=["mergeid", "int_year"], ascending=[True, True])
# On the sorted data set (which should be called dat), how to generate a new variable called

In [117]:
dat.head(10)

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,dn028_1,dn028_2,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year
4,DE-000132-01,2009.0,3.0,2,1.0,1953.0,56.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,1971.0,1973.0,1979.0,1993.0,1997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1973.0,1979.0,1987.0,1997.0,9997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,NaN,NaN,1985.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1986.0,NaN,NaN,NaN,NaN,NaN

In [118]:
# Calculate cumulative sum for 'care_in_year' within each 'mergeid' group
dat["care_experience_"] = (
    dat.groupby("mergeid")["care_in_year"]
    .cumsum()
    .where(dat["care_in_year"] >= 0, np.nan)
)

dat["care_experience"] = (
    dat.groupby("mergeid")["ever_cared_parents"]
    .cumsum()
    .where(dat["ever_cared_parents"] >= 0, np.nan)
)

In [119]:
table(dat["care_experience"])

care_experience
0.0    4271
1.0     794
2.0     205
3.0      57
4.0      21
5.0       3
6.0       1
Name: Count, dtype: int64

In [120]:
dat["care_experience"].mean()

0.2765321375186846

In [121]:
# Fill in missing years between consecutive interviews where care_parents is 1
dat["care_experience"] = (
    dat.groupby("mergeid")
    .apply(
        lambda group: group["care_experience"].interpolate(
            method="linear",
            limit_direction="both",
        ),
    )
    .reset_index(drop=True)  # Reset index to match the original DataFrame
    .fillna(0)
    .astype(int)
)

table(dat["care_experience"])

care_experience
0.0    1365
1.0     256
2.0      66
3.0      19
4.0       5
Name: Count, dtype: int64

In [122]:
dat[["mergeid", "int_year", "care_in_year", "care_experience"]]

,mergeid,int_year,care_in_year,care_experience
4,DE-000132-01,2009.0,0,0.0
5,DE-000132-01,2011.0,0,0.0
6,DE-000132-01,2013.0,0,0.0
7,DE-000132-01,2015.0,0,0.0
8,DE-000132-01,2017.0,0,0.0
...,...,...,...,...
26583,DE-999404-02,2017.0,0,NaN
26585,DE-999990-01,2013.0,0,NaN
26586,DE-999990-01,2015.0,0,NaN
26587,DE-999990-01,2017.0,0,NaN


In [123]:
# outside the household
condition = (
    ((dat["sp009_1"] == 1) & (dat["sp010d1_1"] == 1))
    | ((dat["sp009_2"] == 1) & (dat["sp010d1_2"] == 1))
    | ((dat["sp009_3"] == 1) & (dat["sp010d1_3"] == 1))
)

condition = (dat["sp009_1"] == 1) | (dat["sp009_2"] == 1) | (dat["sp009_3"] == 1)

dat[condition].shape

# need to drop personal (intensive care) INSIDE the houeshold to any other than parent
# need to add variables sp019d1
# rename in waves 4, 5 --> sp/sn above
# variables to add

(56, 308)

In [124]:
# sp011_1: how often inside

condition = [
    ((dat["sp011_1"] >= 2) | (dat["sp011_2"] >= 2) | (dat["sp011_3"] >= 2))
    & (
        (dat["sp009_1"].isin([2, 3]))  # to whom: mother or father
        | (dat["sp009_2"].isin([2, 3]))  # to whom: mother or father
        | (dat["sp009_3"].isin([2, 3]))  # to whom: mother or father
    )
    & (dat["sp018_"] != 1)  # no personal care in hh
    & ((dat["sp019d2"] != 1) & (dat["sp019d3"] != 1)),
]

choice = [1]  # Assign 1 if the conditions are met

dat["light_care"] = np.select(condition, choice, default=0)

In [125]:
table(dat["sp011_1"])

sp011_1
-2.0      1
-1.0      1
 1.0    412
 2.0    641
 3.0    382
 4.0    499
Name: Count, dtype: int64

In [126]:
table(dat["light_care"])

# 0    5281
# 1    1542

light_care
0    6343
1     480
Name: Count, dtype: int64

In [127]:
conditions = [
    (
        ((dat["sp011_1"] == 1) | (dat["sp011_2"] == 1) | (dat["sp011_3"] == 1))
        & (
            (dat["sp009_1"].isin([2, 3]))  # to whom: mother or father
            | (dat["sp009_2"].isin([2, 3]))  # to whom: mother or father
            | (dat["sp009_3"].isin([2, 3]))  # to whom: mother or father
        )
    )
    | (dat["sp018_"] == 1)  # or personal care in hh
    & (
        (dat["sp019d2"] == 1) | (dat["sp019d3"] == 1)
    ),  # include mother and father in law?
    ((dat["sp011_1"].isna()) | (dat["sp011_2"].isna()) | (dat["sp011_3"].isna()))
    & (dat["sp018_"].isna()),
]


choices = [1, np.nan]  # Assign 1 if the conditions are met
choices = [1, 0]  # Assign 1 if the conditions are met


dat["intensive_care"] = np.select(conditions, choices, default=0)

In [128]:
table(dat["intensive_care"])

intensive_care
0    6522
1     301
Name: Count, dtype: int64

In [129]:
table(dat["sp019d2"])

sp019d2
0.0    271
1.0     64
Name: Count, dtype: int64

In [130]:
table(dat["sp018_"] == 1)

sp018_
False    6488
True      335
Name: Count, dtype: int64

## Age mother and mother alive

Parental information Individuals give information on parental age, health and distance individually
by parent. We use this information plainly as given. If individuals respond in several waves we impute
missing parental information given information in periods around the missing data point.

In [131]:
# Define conditions and choices for np.select
conditions_dn026 = [
    (dat["dn026_1"] == 1),
    (dat["dn026_1"] == 5),
]

choices_dn026 = [1, 0]

# Create 'mother_alive' based on 'dn026_1' using np.select
dat["mother_alive"] = np.select(conditions_dn026, choices_dn026, default=np.nan)

# Rename 'dn028_1' to 'age_mother'
dat = dat.rename(columns={"dn028_1": "age_mother"})

In [132]:
table(dat["mother_alive"])

mother_alive
0.0    2732
1.0    1811
Name: Count, dtype: int64

In [133]:
dat["mother_alive"].isna().sum()

2280

In [134]:
dat["age_mother"].isna().sum()

6272

In [135]:
dat["age_mother"].notna().sum()

551

In [136]:
dat["age_mother"].mean()

84.15063520871144

In [137]:
table(dat["mother_alive"])

mother_alive
0.0    2732
1.0    1811
Name: Count, dtype: int64

In [138]:
# Handle negative values in 'dn033_1' and convert to 0 for Excellent, 1 for Very good, and 2 for the rest
conditions_dn033 = [
    (dat["dn033_1"] == 1) | (dat["dn033_1"] == 2),
    (dat["dn033_1"] == 3) | (dat["dn033_1"] == 4),
    (dat["dn033_1"] == 5),
]

choices_dn033 = [0, 1, 2]

# Create 'health_mother' based on 'dn033_1' using np.select
dat["health_mother"] = np.select(conditions_dn033, choices_dn033, default=np.nan)

# Rename 'health_mother_3' to 'health_mother'
dat = dat.rename(columns={"health_mother_3": "health_mother"})

# Re-map values to 0=good, 1=medium, 2=bad
# dat["health_mother"] = dat["health_mother"].replace({0: 0, 1: 1, 2: 3})

In [139]:
table(dat["dn033_1"] > 0)

dn033_1
False    4989
True     1834
Name: Count, dtype: int64

In [140]:
table(dat["health_mother"])

# health_mother
# 0.0     179
# 1.0    1209
# 2.0     446

health_mother
0.0     179
1.0    1209
2.0     446
Name: Count, dtype: int64

In [141]:
# Handle negative values in 'dn026_2' and create 'father_alive'
conditions_dn026_2 = [
    (dat["dn026_2"] == 1),
    (dat["dn026_2"] == 5),
]

choices_dn026_2 = [1, 0]

dat["father_alive"] = np.select(conditions_dn026_2, choices_dn026_2, default=np.nan)

# Rename 'dn028_2' to 'age_father'
dat = dat.rename(columns={"dn028_2": "age_father"})

# Handle negative values in 'dn033_2' and create 'health_father_3'
conditions_dn033_2 = [
    (dat["dn033_2"] == 1) | (dat["dn033_2"] == 2),
    (dat["dn033_2"] == 3) | (dat["dn033_2"] == 4),
    (dat["dn033_2"] == 5),
]

choices_dn033_2 = [0, 1, 2]

dat["health_father_3"] = np.select(conditions_dn033_2, choices_dn033_2, default=np.nan)

In [142]:
table(dat["dn033_2"])

dn033_2
-2.0      2
-1.0     12
 1.0     15
 2.0     47
 3.0    171
 4.0    222
 5.0    159
Name: Count, dtype: int64

In [143]:
table(dat["dn033_2"] > 0)

dn033_2
False    6209
True      614
Name: Count, dtype: int64

In [144]:
table(dat["health_father_3"])

health_father_3
0.0     62
1.0    393
2.0    159
Name: Count, dtype: int64

In [145]:
table(dat["father_alive"])

father_alive
0.0    3380
1.0     612
Name: Count, dtype: int64

In [146]:
# Handle negative values in 'dn030_1' and 'dn030_2', and create 'dist_father' and 'dist_mother'
dat["dist_father"] = dat["dn030_2"].apply(lambda x: x if x >= 0 else np.nan)
dat["dist_mother"] = dat["dn030_1"].apply(lambda x: x if x >= 0 else np.nan)

# Create 'parents_live_close' based on distance criteria using np.select
conditions_distance = [
    (dat["dist_father"] <= 4) | (dat["dist_mother"] <= 4),
]

choices_distance = [1]

dat["parents_live_close"] = np.select(conditions_distance, choices_distance, default=0)

In [147]:
table(dat["parents_live_close"])

parents_live_close
0    6016
1     807
Name: Count, dtype: int64

In [148]:
dat["ep002_"] = dat["ep002_"].apply(lambda x: x if x >= 0 else np.nan)

In [149]:
# Create 'worked_last_period' based on conditions
dat["worked_last_period"] = np.where((dat["ep005_"] == 2) | (dat["ep002_"] == 1), 1, 0)

In [150]:
table(dat["worked_last_period"])

worked_last_period
0    3701
1    3122
Name: Count, dtype: int64


Variable Name
ep005_
Variable Label
Current job situation
Dataset
sharew5_rel7-1-0_ep
Variable Type
directly measured

Representation Type
Categories


-2	Refusal
-1	Don't know
1	Retired
2	Employed or self-employed (including working for family business)
3	Unemployed
4	Permanently sick or disabled
5	Homemaker
97	Other

In [151]:
# EP141_ChangeInJob
# EP125_ContWork
# EP006_EverWorked

In [152]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Shift the 'ep005_' and 'ep002_' variables by one period
dat["lagged_ep005_"] = dat.groupby("mergeid")["ep005_"].shift(1)
dat["lagged_ep002_"] = dat.groupby("mergeid")["ep002_"].shift(1)

# Create 'worked_last_period' based on the lagged values
dat["worked_last_period"] = np.where(
    (dat["lagged_ep005_"] == 2) | (dat["ep002_"] == 1),
    1,
    0,
)
# dat["worked_last_period"] = np.where((dat["lagged_ep005_"] == 2), 1, 0)

# Drop the lagged columns if not needed
dat = dat.drop(["lagged_ep005_", "lagged_ep002_"], axis=1)

In [153]:
table(dat["worked_last_period"])

worked_last_period
0    4738
1    2085
Name: Count, dtype: int64

# Fix initial condition!!!

# death of parent since last period


In [154]:
# low share of parent alive in Fischer
# age == nan --> parent dead?
# or can the come "back alive" if nan means simply just missing

In [155]:
# dn127_1 (mother) dn127_2 (father)
# only since wave 6

# could use age / health of mothers to check this
# if data about age / health of mother in period before and now not, assume
# that mother died
# same for father

In [156]:
# Group the data by 'age' and count missing values in 'age_mother'
missing_age_mother = dat[dat["age_mother"].isna()].groupby("age")["age"].count()

# Display the result
print(missing_age_mother)

age
55.0    387
56.0    382
57.0    441
58.0    473
59.0    435
60.0    405
61.0    488
62.0    440
63.0    465
64.0    507
65.0    479
66.0    483
67.0    419
68.0    468
Name: age, dtype: int64


In [157]:
mask = (dat["age_mother"].isna()) & (dat["mother_alive"] == 1)
dat[mask]

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period
5,DE-000132-01,2011.0,6.0,2,1.0,1953.0,58.0,2,1.0,1953.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,4,NaN,38.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [158]:
table(dat["age_mother"])

age_mother
67.0     1
71.0     1
72.0     1
73.0     2
74.0     2
75.0    12
76.0     9
77.0    18
78.0    24
79.0    28
80.0    30
81.0    43
82.0    41
83.0    39
84.0    46
85.0    40
86.0    43
87.0    39
88.0    27
89.0    24
90.0    25
91.0    14
92.0    11
93.0    14
94.0     6
95.0     5
96.0     2
97.0     1
98.0     3
Name: Count, dtype: int64

In [159]:
dat["freq_visits_mother"] = dat["dn032_1"]
dat["freq_visits_father"] = dat["dn032_2"]

In [160]:
table(dat["freq_visits_father"]).sum()

604

In [161]:
# Create 'mother_alive_2' based on 'mother_alive'
dat["mother_alive_2"] = np.where(dat["mother_alive"] == 1, 1, np.nan)

In [162]:
##

In [163]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])


# Create 'lagged_age_mother' using 'shift' to represent the previous period's values
dat["lagged_age_mother"] = dat.groupby("mergeid")["age_mother"].shift(1)

# Create 'mother_dead' based on the specified conditions
dat["mother_dead"] = np.where(
    dat["age_mother"].isna() & (dat["lagged_age_mother"] > 0),
    1,
    np.nan,
)

In [164]:
table(dat["lagged_age_mother"])

lagged_age_mother
67.0     1
71.0     1
72.0     1
73.0     2
74.0     1
75.0     9
76.0     6
77.0    12
78.0    15
79.0    17
80.0    19
81.0    29
82.0    29
83.0    27
84.0    29
85.0    28
86.0    33
87.0    24
88.0    22
89.0    16
90.0    18
91.0     9
92.0    10
93.0    13
94.0     4
95.0     3
96.0     2
97.0     1
98.0     2
Name: Count, dtype: int64

In [165]:
table(dat["mother_dead"])

mother_dead
1.0    377
Name: Count, dtype: int64

In [166]:
dat["lagged_mother_alive"] = dat.groupby("mergeid")["mother_alive"].shift(1)

# Create 'mother_dead' based on conditions using np.select
conditions = [
    (dat["lagged_mother_alive"] == 0),
    (dat["lagged_mother_alive"] == 1),
]

choices = [1, 0]  # 1 for True, 0 for False

dat["mother_dead_since_last"] = np.select(conditions, choices, np.nan)

In [167]:
table(dat["lagged_mother_alive"])

lagged_mother_alive
0.0    1588
1.0    1173
Name: Count, dtype: int64

In [168]:
table(dat["mother_dead_since_last"])

mother_dead_since_last
0.0    1173
1.0    1588
Name: Count, dtype: int64

In [169]:
dat["mother_dead_since_last"].describe()

count    2761.000000
mean        0.575154
std         0.494409
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: mother_dead_since_last, dtype: float64

## More age info?

In [170]:
# Create 'mother_alive' based on 'dn026_1' using np.select
dat["mother_alive"] = np.select(conditions_dn026, choices_dn026, default=np.nan)

In [171]:
table(dat["mother_alive"])

mother_alive
0.0    2732
1.0    1811
Name: Count, dtype: int64

In [172]:
# Sort the data by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Group the data by 'mergeid' and transform to get the first non-NaN value of 'age_mother'
dat["age_mother_first"] = dat.groupby("mergeid")["age_mother"].transform("first")
dat["int_year_mother_first"] = dat.groupby("mergeid")["int_year"].transform("first")

In [173]:
# Calculate the first non-NaN value in 'age_mother_first' within each group
first_age_mother = dat.groupby("mergeid")["age_mother_first"].transform("first")

# Create 'birth_year_mother' based on the calculation
dat["age_year_mother_new"] = (
    dat["int_year"] - dat["int_year_mother_first"] + first_age_mother
)
# dat['age_mother_full'] = dat['int_year'] + (first_age_mother)

# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])


def first_non_empty(series):
    # Helper function to get the first non-empty value
    return series.dropna().iloc[0] if not series.dropna().empty else np.nan


# Create 'int_year_mother_first' to record the 'int_year' when 'age_mother_first' is first non-empty
dat["int_year_mother_first"] = dat.groupby("mergeid")["int_year"].transform(
    lambda x: first_non_empty(dat["int_year"].where(~dat["age_mother_first"].isna()))
)

In [174]:
# Group the data by 'mergeid'
grouped = dat.groupby("mergeid")

# Determine the most common non-empty value in 'dn027_1' for each 'mergeid'
most_common_value = grouped["dn027_1"].apply(
    lambda x: x.dropna().mode().iloc[0] if not x.dropna().empty else np.nan,
)

# Assign the most common value to all rows within the 'mergeid' group
dat["age_mother_death"] = dat["mergeid"].map(most_common_value)

# Fill any remaining NaN values with np.nan
dat["age_mother_death"].fillna(np.nan, inplace=True)

In [175]:
dat = dat.sort_values(by=["mergeid", "int_year"])

# Initialize an auxiliary variable 'death_transition' to track the transition from 1 to 0
dat["death_transition"] = (dat["mother_alive"] == 0) & (
    dat.groupby("mergeid")["mother_alive"].shift(1) == 1
)

# Calculate 'year_mother_death' based on the first transition from 1 to 0 within each 'mergeid'
dat["year_mother_death"] = dat.groupby("mergeid")["int_year"].transform(
    lambda x: x.where(dat["death_transition"]).min(),
)

# Fill remaining NaN values in 'year_mother_death' with np.nan
# dat["year_mother_death"].fillna(np.nan, inplace=True)


# Identify the first observation in the panel for each 'mergeid'
# first_observation_mask = (
#    dat.groupby("mergeid")["int_year"].transform("first") == dat["int_year"]
# )
#
## Further filter for rows where 'mother_alive' is 0
# first_observation_mother_alive_zero_mask = first_observation_mask & (
#    dat["mother_alive"] == 0
# )
#
## Replace values in 'year_mother_death' with 'int_year - 1' for the first observations
# dat.loc[first_observation_mother_alive_zero_mask, "year_mother_death"] = (
#    dat["int_year"] - 1
# )

In [176]:
# Assuming 'dat' is a pandas DataFrame

# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Identify the first observation in the panel for each 'mergeid'
first_observation_mask = (
    dat.groupby("mergeid")["int_year"].transform("first") == dat["int_year"]
)

# Identify the next observation in the panel for each 'mergeid'
next_observation_mask = (
    dat.groupby("mergeid")["int_year"].transform("first") == dat["int_year"] + 1
)

# Filter for rows where 'mother_alive' is NaN and the conditions are met
# nan_mother_alive_mask = dat["mother_alive"].isna() & first_observation_mask
nan_mother_alive_mask = (
    dat["mother_alive"].isna() & first_observation_mask & next_observation_mask
)

# Replace 'mother_alive' with 1 for the specified rows
dat.loc[nan_mother_alive_mask & (dat["mother_alive"].shift(1) == 1), "mother_alive"] = 1

# Replace 'mother_alive' with 0 for the specified rows
dat.loc[
    nan_mother_alive_mask & (dat["mother_alive"].shift(1) == 0),
    "mother_alive",
] = 0

In [177]:
first_occurrence_condition = (
    (
        (dat["age_year_mother_new"].notna() & dat["age_mother_death"].notna())
        & (dat["age_year_mother_new"] > dat["age_mother_death"])
    )
    .groupby(dat["mergeid"])
    .idxmax()
)
# Identify the next occurrence of "mother_alive == 1" per "mergeid"
next_occurrence_condition = (dat["mother_alive"] != 1).groupby(dat["mergeid"]).shift(-1)
# Replace 'mother_alive' with 0 for rows where it is NaN and the conditions are met
dat.loc[
    (dat["mother_alive"].isna())
    & (dat.index.isin(first_occurrence_condition))
    & (next_occurrence_condition),
    "mother_alive",
] = 0

In [178]:
# # Sort the DataFrame by 'mergeid' and 'int_year'
# dat = dat.sort_values(by=["mergeid", "int_year"])
#
#
# # Create a mask for rows where 'mother_alive' is NaN and the preceding row's 'mother_alive' is 0
# nan_mother_alive_mask = dat["mother_alive"].isna() & (dat["mother_alive"].shift(1) == 0)
#
# # Set 'mother_alive' to 0 for the identified rows
# dat.loc[nan_mother_alive_mask, "mother_alive"] = 0


# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])


# Define a custom function to handle grouping within 'mergeid'
def custom_condition(group):
    return group["mother_alive"].isna() & (group["mother_alive"].shift(1) == 0)


# Apply the custom function within each 'mergeid'
nan_mother_alive_mask = dat.groupby("mergeid").apply(custom_condition)

# Flatten the result to a boolean array
nan_mother_alive_mask = nan_mother_alive_mask.values

# Set 'mother_alive' to 0 for the identified rows
dat.loc[nan_mother_alive_mask, "mother_alive"] = 0

In [179]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Identify the next occurrence of "mother_alive == 1" per "mergeid"
next_occurrence_condition = dat.groupby("mergeid")["mother_alive"].shift(-1) == 1

# Replace 'mother_alive' with 1 for rows where it is NaN and the next occurrence condition is met
dat.loc[(dat["mother_alive"].isna()) & next_occurrence_condition, "mother_alive"] = 1

In [180]:
table(dat["mother_alive"]), dat["mother_alive"].describe()

(mother_alive
 0.0    4478
 1.0    1909
 Name: Count, dtype: int64,
 count    6387.000000
 mean        0.298888
 std         0.457807
 min         0.000000
 25%         0.000000
 50%         0.000000
 75%         1.000000
 max         1.000000
 Name: mother_alive, dtype: float64)

In [181]:
# dat["birth_year_mother"] = (
#     dat.groupby("mergeid")
#     .apply(lambda group: group["year_mother_death"] - group["age_mother_death"])
#     .reset_index(drop=True)
# )

# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Identify the rows where 'mother_alive' switches from 1 to 0 within each 'mergeid'
switch_condition = (dat["mother_alive"] == 1) & (dat["mother_alive"].shift(-1) == 0)

# Calculate 'birth_year_mother' based on 'int_year' and 'age_mother_death' for the switching year
dat.loc[switch_condition, "birth_year_mother"] = (
    dat["int_year"] - dat["age_mother_death"]
)

# Forward-fill the values within each 'mergeid' group
dat["birth_year_mother"] = dat.groupby("mergeid")["birth_year_mother"].ffill()
dat["birth_year_mother"] = dat.groupby("mergeid")["birth_year_mother"].bfill()

In [182]:
# dat["age_year_mother_new"] = dat["age_year_mother_new"].fillna(
#     dat["int_year"] - dat["birth_year_mother"]
# )

dat["age_year_mother_new"] = dat.apply(
    lambda row: row["int_year"] - row["birth_year_mother"]
    if row["mother_alive"] == 1
    else np.nan,
    axis=1,
)

# dat["age_year_mother_new"] = dat["int_year"] - dat["birth_year_mother"][dat["mother_alive"] == 1]

In [183]:
table(dat["age_mother"]).sum()

551

In [184]:
table(dat["mother_alive"])

mother_alive
0.0    4478
1.0    1909
Name: Count, dtype: int64

In [185]:
dat["age_mother"] = dat["age_mother"].fillna(dat["age_year_mother_new"])

In [186]:
table(dat["age_mother"]).sum()

811

In [187]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Group by 'mergeid' and use transform to create 'first_mother_death'
dat["first_mother_death"] = dat.groupby("mergeid")["age_mother_death"].transform(
    lambda x: x.first_valid_index(),
)

In [188]:
# Sort the DataFrame by 'mergeid' and 'int_year'
dat = dat.sort_values(by=["mergeid", "int_year"])

# Identify the rows where 'mother_alive' switches from 1 to 0 within each 'mergeid'
switch_condition = (dat["mother_alive"] == 1) & (dat["mother_alive"].shift(-1) == 0)

# Calculate 'birth_year_mother' based on 'int_year' and 'age_mother_death' for the switching year
dat.loc[switch_condition, "first_age_mother_death"] = (
    dat["int_year"] - dat["age_mother_death"]
)

# Forward-fill the values within each 'mergeid' group
dat["birth_year_mother"] = dat.groupby("mergeid")["birth_year_mother"].ffill()
dat["birth_year_mother"] = dat.groupby("mergeid")["birth_year_mother"].bfill()

In [189]:
dat["first_mother_death"]

4            4.0
5            4.0
6            4.0
7            4.0
8            4.0
          ...   
26583    26577.0
26585    26585.0
26586    26585.0
26587    26585.0
26588    26585.0
Name: first_mother_death, Length: 6823, dtype: float64

In [190]:
dat.loc[
    dat.index[0:30],
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_mother",
        "age_year_mother_new",
        "age_mother_death",
        # "year_mother_death",
        "mother_alive",
        # "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
]

,mergeid,int_year,birth_year_mother,age_mother_first,int_year_mother_first,age_mother,age_year_mother_new,age_mother_death,mother_alive
4,DE-000132-01,2009.0,1918.0,NaN,2009.0,91.0,91.0,97.0,1.0
5,DE-000132-01,2011.0,1918.0,NaN,2009.0,93.0,93.0,97.0,1.0
6,DE-000132-01,2013.0,1918.0,NaN,2009.0,95.0,95.0,97.0,1.0
7,DE-000132-01,2015.0,1918.0,NaN,2009.0,97.0,97.0,97.0,1.0
8,DE-000132-01,2017.0,1918.0,NaN,2009.0,NaN,NaN,97.0,0.0
11,DE-000554-01,2017.0,NaN,NaN,2017.0,NaN,NaN,NaN,1.0
12,DE-000554-01,2020.0,NaN,NaN,2017.0,NaN,NaN,NaN,1.0
16,DE-000802-01,2013.0,NaN,NaN,2013.0,NaN,NaN,87.0,0.0
17,DE-000802-01,2015.0,NaN,NaN,2013.0,NaN,NaN,87.0,0.0
18,DE-000802-01,2017.0,NaN,NaN,2013.0,NaN,NaN,87.0,0.0


In [191]:
2004 - 1915

89

In [192]:
2013 - 1915

98

In [193]:
dat.loc[
    (dat["mother_alive"] == 1) & (dat["age_mother"].isna()),
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_year_mother_new",
        "age_mother_death",
        "age_mother",
        # "year_mother_death",
        # "mother_alive",
        # "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
].head()

,mergeid,int_year,birth_year_mother,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,age_mother
11,DE-000554-01,2017.0,NaN,NaN,2017.0,NaN,NaN,NaN
12,DE-000554-01,2020.0,NaN,NaN,2017.0,NaN,NaN,NaN
111,DE-002935-02,2015.0,NaN,82.0,2013.0,NaN,NaN,NaN
112,DE-002935-02,2017.0,NaN,82.0,2013.0,NaN,NaN,NaN
113,DE-002935-02,2019.0,NaN,82.0,2013.0,NaN,NaN,NaN


In [194]:
table(dat["mother_alive"])

mother_alive
0.0    4478
1.0    1909
Name: Count, dtype: int64

In [195]:
# does not change anything :)
dat.loc[dat["dn027_1"] >= 0, "mother_alive"] = 0

In [196]:
table(dat["mother_alive"])

mother_alive
0.0    4478
1.0    1909
Name: Count, dtype: int64

## Missing age info!!

Use age_mother_first, copy paste into missing years. Construct 

age_mother = first_int_year - age_mother_first

In [197]:
dat["mother_alive"].isna()

4        False
5        False
6        False
7        False
8        False
         ...  
26583    False
26585    False
26586    False
26587    False
26588    False
Name: mother_alive, Length: 6823, dtype: bool

In [198]:
dat["birth_year_mother_new_2"] = dat.apply(
    lambda row: row["int_year_mother_first"] - row["age_mother_first"]
    if (row["mother_alive"] == 1)
    # if ((row["mother_alive"] == 1) | np.isnan(row["mother_alive"]))
    else np.nan,
    axis=1,
)

dat["age_mother_new_2"] = dat["int_year"] - dat["birth_year_mother_new_2"]

In [199]:
dat["birth_year_mother_new_2"].notna().sum()

1095

In [200]:
dat["age_mother_new_2"].notna().sum()

1095

In [201]:
dat.loc[
    (dat["mother_alive"] == 1)
    & (dat["age_mother"].isna())
    & (dat["death_transition"] == True),
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_year_mother_new",
        "age_mother_death",
        "age_mother_new_2",
        # "age_mother"
        "year_mother_death",
        # "mother_alive",
        "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
]
# .iloc[600:800]

,mergeid,int_year,birth_year_mother,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,age_mother_new_2,year_mother_death,death_transition


In [202]:
# aaae(dat["first_int_year"], dat["int_year_mother_first"])

In [203]:
dat.loc[(dat["mother_alive"] == 1) & (dat["age_mother"].isna())]

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period,freq_visits_mother,freq_visits_father,mother_alive_2,lagged_age_mother,mother_dead,lagged_mother_alive,mother_dead_since_last,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,death_transition,year_mother_death,birth_year_mother,first_mother_death,first_age_mother_death,birth_year_mother_new_2,age_mother_new_2
11,DE-000554-01,2017.0,4.0,2,10.0,1961.0,55.0,2,10.0,1961.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,4.0,NaN,NaN,84.0,NaN,NaN,7.0,NaN,NaN,3.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,7,NaN,NaN,NaN,NaN,1978.0,1982.0,

In [204]:
dat["age_mother"] = dat["age_mother"].fillna(dat["age_mother_new_2"])

In [205]:
dat["age_mother"].notna().sum()

1249

In [206]:
dat.loc[(dat["mother_alive"] == 1) & (dat["age_mother"].isna())]

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period,freq_visits_mother,freq_visits_father,mother_alive_2,lagged_age_mother,mother_dead,lagged_mother_alive,mother_dead_since_last,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,death_transition,year_mother_death,birth_year_mother,first_mother_death,first_age_mother_death,birth_year_mother_new_2,age_mother_new_2
11,DE-000554-01,2017.0,4.0,2,10.0,1961.0,55.0,2,10.0,1961.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,4.0,NaN,NaN,84.0,NaN,NaN,7.0,NaN,NaN,3.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,7,NaN,NaN,NaN,NaN,1978.0,1982.0,

In [207]:
(dat["age_mother"] > 0).sum()

1245

In [208]:
table(dat["mother_alive"])

mother_alive
0.0    4478
1.0    1909
Name: Count, dtype: int64

In [209]:
# create ever dead column?
# or replace death_transition with nan if never 0 or 1 observed

## parental health 

In [210]:
# Handle negative values in 'dn033_1' and convert to 0 for Excellent, 1 for Very good, and 2 for the rest
conditions_dn033 = [
    (dat["dn033_1"] == 1) | (dat["dn033_1"] == 2),
    (dat["dn033_1"] == 3) | (dat["dn033_1"] == 4),
    (dat["dn033_1"] == 5),
]

choices_dn033 = [0, 1, 2]

# Create 'health_mother' based on 'dn033_1' using np.select
dat["health_mother"] = np.select(conditions_dn033, choices_dn033, default=np.nan)

# Rename 'health_mother_3' to 'health_mother'
dat = dat.rename(columns={"health_mother_3": "health_mother"})

# Re-map values to 0=good, 1=medium, 2=bad
# dat["health_mother"] = dat["health_mother"].replace({0: 0, 1: 1, 2: 3})

In [211]:
table(dat["health_mother"])

health_mother
0.0     179
1.0    1209
2.0     446
Name: Count, dtype: int64

In [212]:
dat.loc[
    (dat["mother_alive"].isna()) & (dat["health_mother"].notna()),
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_year_mother_new",
        "age_mother_death",
        "age_mother_new_2",
        # "age_mother"
        "year_mother_death",
        # "mother_alive",
        "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
]

,mergeid,int_year,birth_year_mother,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,age_mother_new_2,year_mother_death,death_transition
2034,DE-075970-02,2011.0,NaN,84.0,2007.0,NaN,NaN,NaN,NaN,False
2163,DE-081716-01,2011.0,NaN,NaN,2007.0,NaN,NaN,NaN,NaN,False
8329,DE-308323-01,2011.0,NaN,NaN,2007.0,NaN,NaN,NaN,NaN,False
9268,DE-346173-02,2011.0,NaN,87.0,2004.0,NaN,NaN,NaN,NaN,False
12417,DE-462433-01,2011.0,NaN,NaN,2007.0,NaN,NaN,NaN,NaN,False
13189,DE-493034-02,2011.0,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,False
14738,DE-549635-02,2011.0,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,False
15797,DE-585295-02,2011.0,NaN,76.0,2004.0,NaN,84.0,NaN,NaN,False
16371,DE-607795-01,2011.0,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,False
16410,DE-609514-02,2011.0,NaN,81.0,2006.0,NaN,86.0,NaN,NaN,False


In [213]:
dat.loc[
    (dat["mother_alive"] == 1)
    & (dat["age_mother"].isna())
    & (dat["health_mother"].isna()),
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_year_mother_new",
        "age_mother_death",
        "age_mother_new_2",
        # "age_mother"
        "year_mother_death",
        # "mother_alive",
        "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
].shape

(36, 10)

In [214]:
# drop those?
dat.loc[
    (dat["mother_alive"] == 1) & (dat["health_mother"].isna()),
    [
        "mergeid",
        "int_year",
        "birth_year_mother",
        "age_mother_first",
        "int_year_mother_first",
        "age_year_mother_new",
        "age_mother_death",
        "age_mother_new_2",
        # "age_mother"
        "year_mother_death",
        # "mother_alive",
        "death_transition",
        # "care_in_year",
        # "dn027_1",
    ],
]

,mergeid,int_year,birth_year_mother,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,age_mother_new_2,year_mother_death,death_transition
4,DE-000132-01,2009.0,1918.0,NaN,2009.0,91.0,97.0,NaN,2017.0,False
351,DE-012572-02,2009.0,1923.0,NaN,2009.0,86.0,92.0,NaN,2017.0,False
515,DE-017922-01,2009.0,NaN,NaN,2009.0,NaN,NaN,NaN,NaN,False
1473,DE-058463-01,2008.0,NaN,83.0,2004.0,NaN,NaN,87.0,NaN,False
1656,DE-063529-01,2008.0,1931.0,NaN,2006.0,77.0,86.0,NaN,2019.0,False
...,...,...,...,...,...,...,...,...,...,...
25806,DE-968477-02,2015.0,NaN,NaN,2008.0,NaN,NaN,NaN,NaN,False
25807,DE-968477-02,2017.0,NaN,NaN,2008.0,NaN,NaN,NaN,NaN,False
25808,DE-968477-02,2020.0,NaN,NaN,2008.0,NaN,NaN,NaN,NaN,False
26034,DE-977341-02,2009.0,1926.0,NaN,2006.0,83.0,87.0,NaN,2015.0,False


# Fill missing info in dist mother

In [215]:
table(dat["dist_mother"])

dist_mother
1.0     63
2.0    130
3.0    232
4.0    309
5.0    452
6.0    242
7.0    242
8.0    131
9.0      9
Name: Count, dtype: int64

In [216]:
dat["dist_mother"].notna().sum()

1810

In [217]:
dat[["mergeid", "int_year", "dist_mother"]]

,mergeid,int_year,dist_mother
4,DE-000132-01,2009.0,NaN
5,DE-000132-01,2011.0,7.0
6,DE-000132-01,2013.0,7.0
7,DE-000132-01,2015.0,7.0
8,DE-000132-01,2017.0,NaN
...,...,...,...
26583,DE-999404-02,2017.0,NaN
26585,DE-999990-01,2013.0,NaN
26586,DE-999990-01,2015.0,NaN
26587,DE-999990-01,2017.0,NaN


In [218]:
dat["dist_mother"] = dat.groupby("mergeid")["dist_mother"].transform(
    lambda x: x.ffill().bfill(),
)
dat["dist_mother"] = np.where(dat["mother_alive"] == 0, np.nan, dat["dist_mother"])

In [219]:
dat["dist_mother"].notna().sum()

2012

In [220]:
dat[["mergeid", "int_year", "mother_alive", "dist_mother"]]

,mergeid,int_year,mother_alive,dist_mother
4,DE-000132-01,2009.0,1.0,7.0
5,DE-000132-01,2011.0,1.0,7.0
6,DE-000132-01,2013.0,1.0,7.0
7,DE-000132-01,2015.0,1.0,7.0
8,DE-000132-01,2017.0,0.0,NaN
...,...,...,...,...
26583,DE-999404-02,2017.0,0.0,NaN
26585,DE-999990-01,2013.0,0.0,NaN
26586,DE-999990-01,2015.0,0.0,NaN
26587,DE-999990-01,2017.0,0.0,NaN


## Working (part time full time)

**Retirement** Individuals are considered retired if they respond to be retired in the question on their
current job situation. In addition, individuals are considered retired if they respond **not to be working**
and respond to be **receiving old age pension benefits**.

**Working**  Individuals are considered part-time employed if they respond to be working and provide
a number of working hours within the 5th to 50th percentile of the distribution of working hours. This
corresponds to 10 to 32 hours per week. Individuals are considered full-time employed if they work
more than the median of hours in the distribution of working hours (more than 32 hours per week).
In the model we consider the mass-points of the distribution at the 25th percentile (20hours per week)
and 75th percentile of the distribution (40 hours per week) for working women as part- and full- time
work.

**Years in retirement** Individuals give information on the time they have spend in retirement. If the
information is missing and individuals are considered retired we use information given in SHARElife
to construct retrospectively the year in which the last job ended

In [221]:
dat["working"] = np.where(dat["ep005_"] == 2, 1, np.where(dat["ep005_"] < 0, np.nan, 0))

In [222]:
dat.sort_values(by=["mergeid", "int_year"], inplace=True)

dat["first_int_year"] = dat.groupby("mergeid")["int_year"].transform("first")
dat["lagged_int_year"] = dat.groupby("mergeid")["int_year"].shift(1)
dat["lagged_working"] = dat.groupby("mergeid")["working"].shift(1)

In [223]:
dat["ep013_"]

4         NaN
5        38.5
6        38.5
7        38.0
8         NaN
         ... 
26583     NaN
26585     8.0
26586     9.0
26587     NaN
26588     NaN
Name: ep013_, Length: 6823, dtype: float64

In [224]:
dat["full_time"] = np.where((dat["working"] == 1) & (dat["ep013_"] > 32), 1, 0)

dat["part_time"] = np.where(
    (dat["working"] == 1) & (dat["ep013_"] >= 10) & (dat["ep013_"] <= 32),
    1,
    0,
)


# # Create the 'full_time' variable based on the first condition
# dat['full_time'] = np.where(dat['ep013_'] > 32, 1, 0)
#
# # Create the 'part_time' variable based on the second condition
# dat['part_time'] = np.where((dat['ep013_'] >= 10) & (dat['ep013_'] <= 32), 1, 0)

In [225]:
conditions = [
    (dat["working"] == 1) & (dat["ep013_"] > 32),
    (dat["working"] == 1) & (dat["ep013_"] >= 10) & (dat["ep013_"] <= 32),
]

values = [1, 0]

# Use numpy.select to create the 'full_time' variable
dat["full_time_nan"] = np.select(conditions, values, np.nan)

In [226]:
dat["full_time"].notna().sum(), dat["working"].notna().sum()

(6823, 6823)

In [227]:
dat.loc[dat["full_time"] == 1].shape

(814, 343)

In [228]:
dat.loc[dat["part_time"] == 1].shape

(870, 343)

In [229]:
dat.loc[dat["ep013_"] > 0].shape

(2151, 343)

In [230]:
# restrospective waves (3 and 7)
##### work experience
# wave 3:
# sl_re011_1: year first job started
# ... sl_re011_20: year 20th job started
# calculate potential work experience with sl_re011_1

In [231]:
dat.shape

(6823, 343)

In [232]:
# Identify columns that start with "sl_re011"
job_start = [col for col in dat.columns if col.startswith("sl_re011")]

# Iterate through columns and set values < 0 to NA
for job in job_start:
    dat[job] = np.where(dat[job] < 0, np.nan, dat[job])

# Identify columns that start with "sl_re026"
job_end = [col for col in dat.columns if col.startswith("sl_re026")]

# Iterate through columns and set values < 0 to NA, and values == 9997 to int_year
for job in job_end:
    dat[job] = np.where(
        dat[job] < 0,
        np.nan,
        np.where(dat[job] == 9997, dat["int_year"], dat[job]),
    )
    # dat[job] = np.where(dat[job] < 0, np.nan,dat[job])

In [233]:
dat["sl_re026_1"].describe()

count    1389.000000
mean     1983.509719
std        15.944905
min      1956.000000
25%      1971.000000
50%      1980.000000
75%      1991.000000
max      2017.000000
Name: sl_re026_1, dtype: float64

In [234]:
dat["sl_re026_1"].isna().sum()

5434

In [235]:
table(dat["ep005_"]), dat["ep005_"].isna().sum()

(ep005_
 1.0     2443
 2.0     2360
 3.0      254
 4.0      223
 5.0      830
 97.0      83
 Name: Count, dtype: int64,
 630)

In [236]:
dat.loc[
    dat.index[0:30],
    [
        "mergeid",
        "int_year",
        "working",
        "sl_re011_1",
        "sl_re026_1",
        "sl_re011_2",
        "sl_re026_2",
        "sl_re011_3",
        "sl_re026_3",
        "sl_re011_4",
        "sl_re026_4",
        "sl_re011_5",
        "sl_re026_5",
        "sl_re011_6",
        "sl_re026_6",
    ],
]

,mergeid,int_year,working,sl_re011_1,sl_re026_1,sl_re011_2,sl_re026_2,sl_re011_3,sl_re026_3,sl_re011_4,sl_re026_4,sl_re011_5,sl_re026_5,sl_re011_6,sl_re026_6
4,DE-000132-01,2009.0,0.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1993.0,1997.0,1997.0,2009.0,NaN,NaN
5,DE-000132-01,2011.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DE-000132-01,2013.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DE-000132-01,2015.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DE-000132-01,2017.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,DE-000554-01,2017.0,1.0,1978.0,1982.0,1982.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,DE-000554-01,2020.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,DE-000802-01,2013.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,DE-000802-01,2015.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,DE-000802-01,2017.0,0.0,1970.0,1971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Most recent job started

In [237]:
# Create a list of column names that start with 'sl_re011_'
sl_re011_columns = [f"sl_re011_{i}" for i in range(1, 21)]


# Function to find the most recent job started
def most_recent_job(row):
    for col in reversed(sl_re011_columns):
        if not pd.isna(row[col]):
            return row[col]
    return np.nan


# Create the 'most_recent_job_started' variable
dat["most_recent_job_started"] = dat.apply(most_recent_job, axis=1)

In [238]:
# Create a list of column names that start with 'sl_re026_'
sl_re026_columns = [f"sl_re026_{i}" for i in range(1, 21)]


# Function to find the most recent job ended
def most_recent_job_ended(row):
    for col in reversed(sl_re026_columns):
        if not pd.isna(row[col]):
            return row[col]
    return np.nan


# Create the 'most_recent_job_ended' variable
dat["most_recent_job_ended"] = dat.apply(most_recent_job_ended, axis=1)

In [239]:
dat["most_recent_job_ended"] = dat.groupby("mergeid")[
    "most_recent_job_ended"
].transform(lambda x: x.ffill().bfill())

In [240]:
dat["most_recent_job_started"] = dat.groupby("mergeid")[
    "most_recent_job_started"
].transform(lambda x: x.ffill().bfill())

In [241]:
dat["most_recent_job_started"]

4        1997.0
5        1997.0
6        1997.0
7        1997.0
8        1997.0
          ...  
26583    2000.0
26585    2006.0
26586    2006.0
26587    2006.0
26588    2006.0
Name: most_recent_job_started, Length: 6823, dtype: float64

In [242]:
table(dat["most_recent_job_ended"])

most_recent_job_ended
1960.0      12
1962.0      17
1963.0      15
1964.0      11
1965.0      16
1966.0      17
1967.0       5
1968.0      24
1969.0      11
1970.0      21
1971.0      19
1972.0      13
1973.0      29
1974.0      26
1975.0      45
1976.0      29
1977.0      15
1978.0      13
1979.0      44
1980.0      17
1981.0      22
1982.0      30
1983.0      26
1984.0      17
1985.0      13
1986.0      21
1987.0      15
1988.0      10
1989.0      22
1990.0      56
1991.0      20
1992.0      49
1993.0      26
1994.0      22
1995.0      58
1996.0      32
1997.0      49
1998.0      52
1999.0      32
2000.0      70
2001.0      72
2002.0     116
2003.0     138
2004.0     108
2005.0     106
2006.0     108
2007.0     100
2008.0     363
2009.0     703
2010.0      53
2011.0      60
2012.0      91
2013.0      75
2014.0     120
2015.0     125
2016.0     144
2017.0    1300
Name: Count, dtype: int64

In [243]:
conditions = [(dat["sl_re011_1"].notna() & (dat["wave"] == 3))]
values = [1]

# Use numpy.select to create the 'wave_3_response' variable
dat["wave_3_response"] = np.select(conditions, values, default=np.nan)


# Define the conditions and corresponding values for wave 7 response
conditions_wave_7 = [(dat["sl_re011_1"].notna() & (dat["wave"] == 7))]
values_wave_7 = [1]

# Use numpy.select to create the 'wave_7_response' variable
dat["wave_7_response"] = np.select(conditions_wave_7, values_wave_7, default=np.nan)

In [244]:
dat["wave_3_response"] = dat.groupby("mergeid")["wave_3_response"].transform(
    lambda x: x.ffill().bfill(),
)
dat["wave_7_response"] = dat.groupby("mergeid")["wave_7_response"].transform(
    lambda x: x.ffill().bfill(),
)

In [245]:
dat["wave_3_response"] = dat["wave_3_response"].fillna(0)
dat["wave_7_response"] = dat["wave_7_response"].fillna(0)

dat["both_wave_3_and_7"] = (
    (dat["wave_3_response"] == 1) & (dat["wave_7_response"] == 1)
).astype(int)

In [246]:
table(dat["both_wave_3_and_7"])

both_wave_3_and_7
0    6823
Name: Count, dtype: int64

# Retrospective work experience

In [247]:
prefixes = ["sl_re011_", "sl_re026_"]

# Iterate over the prefixes and apply forward and backward fill
for prefix in prefixes:
    relevant_cols = [col for col in dat.columns if col.startswith(prefix)]
    dat[relevant_cols] = dat.groupby("mergeid")[relevant_cols].transform(
        lambda x: x.ffill().bfill(),
    )

In [248]:
dat["job_just_started"] = 0

dat.loc[
    dat["most_recent_job_started"].notna() & dat["most_recent_job_ended"].isna(),
    "job_just_started",
] = 1

dat["job_just_ended"] = 0

dat.loc[
    dat["most_recent_job_ended"].notna() & dat["most_recent_job_started"].isna(),
    "job_just_ended",
] = 1

In [249]:
table(dat["job_just_started"])

job_just_started
0    6817
1       6
Name: Count, dtype: int64

In [250]:
table(dat["job_just_ended"])

job_just_ended
0    6823
Name: Count, dtype: int64

In [251]:
dat.loc[dat["job_just_started"] == 1][
    [
        "mergeid",
        "int_year",
        "age",
        "working",
        "sl_re011_1",
        "sl_re026_1",
        "sl_re011_2",
        "sl_re026_2",
        "sl_re011_3",
        "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        "sl_re011_5",
        "sl_re026_5",
        "sl_re011_6",
        "sl_re026_6",
        "most_recent_job_started",
        "most_recent_job_ended",
        # "wave_3_response",
        # "wave_7_response"
    ]
]

,mergeid,int_year,age,working,sl_re011_1,sl_re026_1,sl_re011_2,sl_re026_2,sl_re011_3,sl_re026_3,sl_re011_5,sl_re026_5,sl_re011_6,sl_re026_6,most_recent_job_started,most_recent_job_ended
9042,DE-337772-02,2013.0,63.0,0.0,1965.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1965.0,NaN
9043,DE-337772-02,2015.0,65.0,0.0,1965.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1965.0,NaN
9044,DE-337772-02,2017.0,68.0,0.0,1965.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1965.0,NaN
11910,DE-444005-03,2015.0,56.0,1.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1974.0,NaN
11911,DE-444005-03,2017.0,58.0,0.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1974.0,NaN
11912,DE-444005-03,2020.0,60.0,0.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1974.0,NaN


In [252]:
dat.loc[
    dat.index[:30],
    [
        "mergeid",
        "int_year",
        "working",
        "sl_re011_1",
        "sl_re026_1",
        "sl_re011_2",
        "sl_re026_2",
        "sl_re011_3",
        "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        "sl_re011_5",
        "sl_re026_5",
        "sl_re011_6",
        "sl_re026_6",
        "most_recent_job_started",
        "most_recent_job_ended",
        # "wave_3_response",
        # "wave_7_response"
    ],
]

,mergeid,int_year,working,sl_re011_1,sl_re026_1,sl_re011_2,sl_re026_2,sl_re011_3,sl_re026_3,sl_re011_5,sl_re026_5,sl_re011_6,sl_re026_6,most_recent_job_started,most_recent_job_ended
4,DE-000132-01,2009.0,0.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1997.0,2009.0,NaN,NaN,1997.0,2009.0
5,DE-000132-01,2011.0,1.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1997.0,2009.0,NaN,NaN,1997.0,2009.0
6,DE-000132-01,2013.0,1.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1997.0,2009.0,NaN,NaN,1997.0,2009.0
7,DE-000132-01,2015.0,1.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1997.0,2009.0,NaN,NaN,1997.0,2009.0
8,DE-000132-01,2017.0,0.0,1971.0,1973.0,1973.0,1979.0,1979.0,1987.0,1997.0,2009.0,NaN,NaN,1997.0,2009.0
11,DE-000554-01,2017.0,1.0,1978.0,1982.0,1982.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,1982.0,2017.0
12,DE-000554-01,2020.0,1.0,1978.0,1982.0,1982.0,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,1982.0,2017.0
16,DE-000802-01,2013.0,1.0,1970.0,1971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,1971.0
17,DE-000802-01,2015.0,1.0,1970.0,1971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,1971.0
18,DE-000802-01,2017.0,0.0,1970.0,1971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,1971.0


In [253]:
# # Define a function to check if the columns are answered in both waves
# def both_waves_answered(row):
#     wave_3_cols = [f'sl_re011_{i}' for i in range(1, 21)]
#     wave_7_cols = [f'sl_re011_{i}' for i in range(1, 21)]
#
#     for col in wave_3_cols + wave_7_cols:
#         if pd.isna(row[col]):
#             return 0
#
#     return 1
#
# # Create the 'answered_both_waves' variable
# dat['answered_both_waves'] = dat.apply(both_waves_answered, axis=1)

By mergeid, fill the nan rows in the columns starting with sl_re026_ and sl_re011_ with the respective non missing values in those columns. E.g. for sl_re011_1 fill the missing cells with the non missing value for sl_re011_1 for that given individual. The sl_re026_ and sl_re011_ values were only answered in certain years.

In [254]:
conditions = [(dat["full_time"] == 1), (dat["part_time"] == 1)]

choices = [1, 0.5]

dat["exp_weight"] = np.select(conditions, choices, default=0)

dat["lagged_exp_weight"] = dat.groupby("mergeid")["exp_weight"].shift(1)

In [255]:
table(dat["sl_re026_1"])

sl_re026_1
1956.0      2
1957.0      3
1958.0     18
1959.0     20
1960.0     57
         ... 
2013.0     20
2014.0     27
2015.0     38
2016.0     26
2017.0    261
Name: Count, Length: 62, dtype: int64

In [256]:
# -2 Refusal
# -1 Don't know
# 9997 Still in this job

In [257]:
# List of columns starting with "sl_re026_" or "sl_re011_"
columns_to_check = [
    col
    for col in dat.columns
    if col.startswith("sl_re026_") or col.startswith("sl_re011_")
]

# Use map with a lambda function to replace both negative and values greater than or equal to 9997 with np.nan
dat[columns_to_check] = dat[columns_to_check].apply(
    lambda x: x.map(lambda val: np.nan if val < 0 else val),
)

In [258]:
table(dat["sl_re026_5"])

sl_re026_5
1969.0     4
1972.0     3
1973.0     4
1974.0     3
1976.0     6
1978.0     5
1979.0    10
1980.0    10
1981.0    10
1982.0     8
1983.0     8
1984.0     4
1985.0    12
1986.0    15
1987.0    17
1988.0     9
1989.0    11
1990.0    22
1991.0    22
1992.0    11
1993.0    34
1994.0    41
1995.0    42
1996.0    27
1997.0    11
1998.0    24
1999.0    20
2000.0    22
2001.0    17
2002.0    11
2003.0    25
2004.0    31
2005.0    15
2006.0    28
2007.0    19
2008.0    50
2009.0    45
2010.0    16
2011.0    12
2012.0     8
2013.0    18
2014.0     8
2015.0    13
2016.0    12
2017.0    87
Name: Count, dtype: int64

In [259]:
# List of suffix numbers (1 to 20)
suffixes = range(1, 21)

# Initialize an empty DataFrame to store the filtered rows
filtered_rows = pd.DataFrame()

# Iterate through the suffixes and filter the rows
for suffix in suffixes:
    column_name = f"sl_re026_{suffix}"
    filtered_rows = pd.concat([filtered_rows, dat[dat[column_name] > 2020]])

# Display the filtered rows
filtered_rows

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period,freq_visits_mother,freq_visits_father,mother_alive_2,lagged_age_mother,mother_dead,lagged_mother_alive,mother_dead_since_last,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,death_transition,year_mother_death,birth_year_mother,first_mother_death,first_age_mother_death,birth_year_mother_new_2,age_mother_new_2,working,first_int_year,lagged_int_year,lagged_working,full_time,part_time,full_time_nan,most_recent_job_started,most_recent_job_ended,wave_3_response,wave_7_response,both_wave_3_and_7,job_just_started,job_just_ended,exp_weight,lagged_exp_weight


In [260]:
# Calculate the exper columns
dat["exper_1"] = np.abs(dat["sl_re026_1"] - dat["sl_re011_1"])
dat["exper_2"] = np.abs(dat["sl_re026_2"] - dat["sl_re011_2"])
dat["exper_3"] = np.abs(dat["sl_re026_3"] - dat["sl_re011_3"])
dat["exper_4"] = np.abs(dat["sl_re026_4"] - dat["sl_re011_4"])
dat["exper_5"] = np.abs(dat["sl_re026_5"] - dat["sl_re011_5"])
dat["exper_6"] = np.abs(dat["sl_re026_6"] - dat["sl_re011_6"])
dat["exper_7"] = np.abs(dat["sl_re026_7"] - dat["sl_re011_7"])
dat["exper_8"] = np.abs(dat["sl_re026_8"] - dat["sl_re011_8"])
dat["exper_9"] = np.abs(dat["sl_re026_9"] - dat["sl_re011_9"])
dat["exper_10"] = np.abs(dat["sl_re026_10"] - dat["sl_re011_10"])
dat["exper_11"] = np.abs(dat["sl_re026_11"] - dat["sl_re011_11"])
dat["exper_12"] = np.abs(dat["sl_re026_12"] - dat["sl_re011_12"])
dat["exper_13"] = np.abs(dat["sl_re026_13"] - dat["sl_re011_13"])
dat["exper_14"] = np.abs(dat["sl_re026_14"] - dat["sl_re011_14"])
dat["exper_15"] = np.abs(dat["sl_re026_15"] - dat["sl_re011_15"])
dat["exper_16"] = np.abs(dat["sl_re026_16"] - dat["sl_re011_16"])
dat["exper_17"] = np.abs(dat["sl_re026_17"] - dat["sl_re011_17"])
dat["exper_18"] = np.abs(dat["sl_re026_18"] - dat["sl_re011_18"])
dat["exper_19"] = np.abs(dat["sl_re026_19"] - dat["sl_re011_19"])
dat["exper_20"] = np.abs(dat["sl_re026_20"] - dat["sl_re011_20"])

In [261]:
dat[["mergeid", "sl_re026_1", "exper_1"]].head(10)

,mergeid,sl_re026_1,exper_1
4,DE-000132-01,1973.0,2.0
5,DE-000132-01,1973.0,2.0
6,DE-000132-01,1973.0,2.0
7,DE-000132-01,1973.0,2.0
8,DE-000132-01,1973.0,2.0
11,DE-000554-01,1982.0,4.0
12,DE-000554-01,1982.0,4.0
16,DE-000802-01,1971.0,1.0
17,DE-000802-01,1971.0,1.0
18,DE-000802-01,1971.0,1.0


In [262]:
import pandas as pd

# Assuming you have already defined the list of suffixes
suffixes = range(1, 17)

# Create an empty list to store DataFrames
selected_dataframes = []

# Loop through each suffix and select rows where sl_re026_{suffix} < sl_re011_{suffix}
for suffix in suffixes:
    condition = dat[f"sl_re026_{suffix}"] < dat[f"sl_re011_{suffix}"]
    selected_suffix_rows = dat[condition]

    # Append the selected DataFrame to the list
    selected_dataframes.append(selected_suffix_rows)

# Concatenate the list of DataFrames into one DataFrame
selected_rows = pd.concat(selected_dataframes, ignore_index=True)

In [263]:
selected_rows[columns_to_check].head()

,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20
0,1965.0,1971.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963.0,1991.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1965.0,1971.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963.0,1991.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1965.0,1971.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963.0,1991.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1965.0,1971.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963.0,1991.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1965.0,1971.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963.0,1991.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
table(dat["exper_1"])

exper_1
0.0     100
1.0     488
2.0     532
3.0     515
4.0     357
5.0     259
6.0     231
7.0     212
8.0     184
9.0     114
10.0    142
11.0    119
12.0    111
13.0     61
14.0     53
15.0     65
16.0     51
17.0     43
18.0     44
19.0     22
20.0     29
21.0     42
22.0     30
23.0     43
24.0     24
25.0     33
26.0     21
27.0     28
28.0     18
29.0     24
30.0     24
31.0     33
32.0     20
33.0     38
34.0     33
35.0     29
36.0     67
37.0     71
38.0     83
39.0     51
40.0     73
41.0     78
42.0     32
43.0     54
44.0     35
45.0     41
46.0     18
47.0     11
48.0     14
49.0      5
50.0      4
51.0      3
Name: Count, dtype: int64

In [265]:
dat["exper_1"].isna().sum()

2011

In [266]:
dat["exper_1"].head(20)

4      2.0
5      2.0
6      2.0
7      2.0
8      2.0
11     4.0
12     4.0
16     1.0
17     1.0
18     1.0
26     1.0
27     1.0
28     1.0
30     NaN
33    10.0
47     NaN
48     NaN
59     2.0
60     2.0
61     2.0
Name: exper_1, dtype: float64

## Compute exper weights

In [267]:
# Create a list of columns to keep (exclude columns starting with 'weight_exper')
columns_to_keep = [col for col in dat.columns if not col.startswith("weight_exper")]

# Select the columns to keep in the DataFrame
dat = dat[columns_to_keep]

In [268]:
# for suffix in range(1, 21):
#     sl_re_column = f'sl_re016_{suffix}'
#     weight_exper_column = f'weight_exper_{suffix}'
#
#     dat[weight_exper_column] = dat[sl_re_column].apply(lambda x: 1 if x == 1 else (0.5 if x == 2 else 0))

suffixes = range(1, 21)

weight_columns = []

for suffix in suffixes:
    sl_re_column = f"sl_re016_{suffix}"
    weight_exper_column = f"weight_exper_{suffix}"

    weight_values = dat[sl_re_column].apply(
        lambda x: 1 if x == 1 else (0.5 if x == 2 else 0),
    )

    weight_columns.append(pd.Series(weight_values, name=weight_exper_column))

dat = pd.concat([dat] + weight_columns, axis=1)

In [269]:
table(weight_columns[0])

weight_exper_1
0.0    5554
0.5      68
1.0    1201
Name: Count, dtype: int64

In [270]:
table(dat["weight_exper_1"])

weight_exper_1
0.0    5554
0.5      68
1.0    1201
Name: Count, dtype: int64

In [271]:
table(dat["weight_exper_1"])

weight_exper_1
0.0    5554
0.5      68
1.0    1201
Name: Count, dtype: int64

In [272]:
# Job was full-time or part-time: sl_re016_{suffix}

# start spell: sl_re011_{suffix}
# stop spell: sl_re026_{suffix}


# year switch to part-time: sl_re018_{suffix}
# year switch to full-time: sl_re020_{suffix}

In [273]:
dat["job_ended"] = np.where(
    dat["first_int_year"] <= dat["most_recent_job_ended"],
    dat["first_int_year"],
    dat["most_recent_job_ended"],
)

In [274]:
dat["job_ended"]

4        2009.0
5        2009.0
6        2009.0
7        2009.0
8        2009.0
          ...  
26583    2002.0
26585    2013.0
26586    2013.0
26587    2013.0
26588    2013.0
Name: job_ended, Length: 6823, dtype: float64

In [275]:
suffixes = range(1, 17)

for suffix in suffixes:
    dat[f"weight_exper_{suffix}"] = np.nan

    job_ended = np.where(
        dat[f"sl_re026_{suffix}"] >= dat["first_int_year"],
        dat["first_int_year"],
        dat[f"sl_re026_{suffix}"],
    )
    # job_ended = dat[f"sl_re026_{suffix}"]

    always_full_time = dat[f"sl_re016_{suffix}"] == 1.0
    dat.loc[always_full_time, f"weight_exper_{suffix}"] = 1.0 * np.abs(
        job_ended - dat[f"sl_re011_{suffix}"],
    )

    always_part_time = dat[f"sl_re016_{suffix}"] == 2.0
    dat.loc[always_part_time, f"weight_exper_{suffix}"] = 0.5 * np.abs(
        job_ended - dat[f"sl_re011_{suffix}"],
    )

    switched_from_full_to_part_time = dat[f"sl_re016_{suffix}"] == 3.0
    dat.loc[switched_from_full_to_part_time, f"weight_exper_{suffix}"] = 1 * np.abs(
        dat[f"sl_re018_{suffix}"] - dat[f"sl_re011_{suffix}"],
    ) + 0.5 * np.abs(job_ended - dat[f"sl_re018_{suffix}"])

    switched_from_part_to_full_time = dat[f"sl_re016_{suffix}"] == 4.0
    dat.loc[switched_from_part_to_full_time, f"weight_exper_{suffix}"] = 0.5 * np.abs(
        dat[f"sl_re018_{suffix}"] - dat[f"sl_re011_{suffix}"],
    ) + 1 * np.abs(job_ended - dat[f"sl_re020_{suffix}"])

In [276]:
# work experience
suffixes = range(1, 17)

# Create a list of column names for 'weight_exper_' columns
weight_columns = [f"weight_exper_{i}" for i in suffixes]

# Calculate work_experience row-wise and store the result in a new column
dat["_retro_work_exp"] = dat[weight_columns].sum(axis=1)

# Group by 'mergeid' and transform to propagate work_experience value
# dat['work_experience'] = dat.groupby('mergeid')['work_experience'].transform('first')

# Calculate the maximum work_experience value within each 'mergeid' group
# max_work_experience = dat.groupby('mergeid')['work_experience'].transform('max')
dat["retro_work_exp"] = dat.groupby("mergeid")["_retro_work_exp"].transform("max")


# Create a copy of the DataFrame to de-fragment it
dat = dat.copy()

In [277]:
dat["retro_work_exp"].sum()

109890.5

In [278]:
dat["_retro_work_exp"].sum()

31790.5

In [279]:
dat["sl_re026_1"].sum()

9549154.0

In [282]:
dat["sl_re026_2"].sum()

7164071.0

In [283]:
dat["sl_re026_3"].sum()

4632673.0

In [284]:
dat["most_recent_job_ended"].sum()

9673401.0

In [280]:
dat.loc[
    dat["exper_1"] == 0,
    [
        "mergeid",
        "int_year",
        "lagged_working",
        "exper_1",
        "weight_exper_1",
        "exper_2",
        "weight_exper_2",
        "most_recent_job_ended",
        "_retro_work_exp",
        "retro_work_exp",
    ],
].tail(33)

,mergeid,int_year,lagged_working,exper_1,weight_exper_1,exper_2,weight_exper_2,most_recent_job_ended,_retro_work_exp,retro_work_exp
17302,DE-646611-01,2015.0,0.0,0.0,NaN,NaN,NaN,1968.0,0.0,0.0
17303,DE-646611-01,2017.0,0.0,0.0,0.0,NaN,NaN,1968.0,0.0,0.0
18125,DE-673853-01,2017.0,NaN,0.0,NaN,1.0,0.5,2017.0,29.5,29.5
18126,DE-673853-01,2019.0,1.0,0.0,NaN,1.0,NaN,2017.0,0.0,29.5
19497,DE-729383-01,2007.0,NaN,0.0,NaN,5.0,NaN,2006.0,0.0,27.5
19498,DE-729383-01,2008.0,0.0,0.0,0.0,5.0,2.5,2006.0,27.5,27.5
19499,DE-729383-01,2011.0,0.0,0.0,NaN,5.0,NaN,2006.0,0.0,27.5
19500,DE-729383-01,2013.0,0.0,0.0,NaN,5.0,NaN,2006.0,0.0,27.5
19501,DE-729383-01,2015.0,0.0,0.0,NaN,5.0,NaN,2006.0,0.0,27.5
20239,DE-756742-01,2017.0,NaN,0.0,0.0,10.0,5.0,2017.0,11.5,11.5


In [281]:
dat.loc[
    dat["first_int_year"] < dat["most_recent_job_ended"],
    [
        "mergeid",
        "int_year",
        "exper_1",
        "weight_exper_1",
        "exper_2",
        "weight_exper_2",
        "most_recent_job_ended",
        "work_experience",
    ],
].head(20)

KeyError: "['work_experience'] not in index"

In [ ]:
table(dat["weight_exper_1"])

In [ ]:
dat[["mergeid", "exper_1", "weight_exper_1"]].head(20)

In [ ]:
table(dat["work_experience"])

In [ ]:
dat[["mergeid", "exper_1", "weight_exper_1", "work_experience"]].head(20)

In [286]:
# List of suffix numbers (1 to 20)
suffixes = range(1, 21)

# Initialize an empty DataFrame to store the filtered rows
filtered_rows = pd.DataFrame()

# Iterate through the suffixes and filter the rows
for suffix in suffixes:
    diff_column = dat[f"sl_re026_{suffix}"] - dat[f"sl_re011_{suffix}"]
    filtered_rows = pd.concat([filtered_rows, dat[diff_column < 0]])

# Create a list of ordered column names
ordered_columns = []
for suffix in suffixes:
    ordered_columns.extend([f"sl_re026_{suffix}", f"sl_re011_{suffix}"])

# Add any other columns you want to include (e.g., 'mergeid' and 'int_year')
ordered_columns = ["mergeid", "int_year"] + ordered_columns

# Create a new DataFrame with the ordered columns
filtered_rows[ordered_columns].shape

(43, 42)

In [287]:
# Create a list of column names that start with 'sl_re011_' and 'weight_exper_'
sl_re011_columns = [f"sl_re011_{i}" for i in range(1, 17)]
weight_exper_columns = [f"weight_exper_{i}" for i in range(1, 17)]


# Function to find the corresponding weight_exper_{suffix} for most_recent_job_started
def corresponding_weight_exper(row):
    most_recent_job_started = row["most_recent_job_started"]
    for i, col in enumerate(sl_re011_columns):
        if not pd.isna(row[col]) and row[col] == most_recent_job_started:
            return row[weight_exper_columns[i]]
    return np.nan


# Create the 'corresponding_weight_exper' variable
dat["corresponding_weight_exper"] = dat.apply(corresponding_weight_exper, axis=1)

In [288]:
prefixes = ["weight_exper_"]

# Iterate over the prefixes and apply forward and backward fill
for prefix in prefixes:
    relevant_cols = [col for col in dat.columns if col.startswith(prefix)]
    dat[relevant_cols] = dat.groupby("mergeid")[relevant_cols].transform(
        lambda x: x.ffill().bfill(),
    )

In [289]:
dat[["most_recent_job_started", "corresponding_weight_exper"]]

,most_recent_job_started,corresponding_weight_exper
4,1997.0,12.0
5,1997.0,NaN
6,1997.0,NaN
7,1997.0,NaN
8,1997.0,NaN
...,...,...
26583,2000.0,NaN
26585,2006.0,NaN
26586,2006.0,NaN
26587,2006.0,3.5


In [290]:
dat["corresponding_weight_exper"] = dat.groupby("mergeid")[
    "corresponding_weight_exper"
].transform(lambda x: x.ffill().bfill())

In [291]:
dat[["most_recent_job_started", "corresponding_weight_exper"]]

,most_recent_job_started,corresponding_weight_exper
4,1997.0,12.0
5,1997.0,12.0
6,1997.0,12.0
7,1997.0,12.0
8,1997.0,12.0
...,...,...
26583,2000.0,2.0
26585,2006.0,3.5
26586,2006.0,3.5
26587,2006.0,3.5


In [292]:
dat[
    [
        "mergeid",
        "sl_re011_5",
        "sl_re026_5",
        "sl_re016_5",
        "weight_exper_5",
        "most_recent_job_ended",
        "corresponding_weight_exper",
    ]
].head(50)

,mergeid,sl_re011_5,sl_re026_5,sl_re016_5,weight_exper_5,most_recent_job_ended,corresponding_weight_exper
4,DE-000132-01,1997.0,2009.0,1.0,12.0,2009.0,12.0
5,DE-000132-01,1997.0,2009.0,NaN,12.0,2009.0,12.0
6,DE-000132-01,1997.0,2009.0,NaN,12.0,2009.0,12.0
7,DE-000132-01,1997.0,2009.0,NaN,12.0,2009.0,12.0
8,DE-000132-01,1997.0,2009.0,NaN,12.0,2009.0,12.0
11,DE-000554-01,NaN,NaN,NaN,NaN,2017.0,35.0
12,DE-000554-01,NaN,NaN,NaN,NaN,2017.0,35.0
16,DE-000802-01,NaN,NaN,NaN,NaN,1971.0,1.0
17,DE-000802-01,NaN,NaN,NaN,NaN,1971.0,1.0
18,DE-000802-01,NaN,NaN,NaN,NaN,1971.0,1.0


In [293]:
dat["working"].isna().sum()  # :)

0

In [294]:
dat["working_cum"] = dat.groupby("mergeid")["working"].cumsum()

In [295]:
table(dat["working_cum"])

working_cum
0.0    3348
1.0    1906
2.0     948
3.0     476
4.0     136
5.0       8
6.0       1
Name: Count, dtype: int64

In [296]:
dat["lagged_int_year"] = dat.groupby("mergeid")["int_year"].shift(1)
dat["lagged_working"] = dat.groupby("mergeid")["working"].shift(1)

In [297]:
# check !

# Sort the DataFrame by 'mergeid' and 'int_year'
dat.sort_values(by=["mergeid", "int_year"], inplace=True)

dat["first_int_year"] = dat.groupby("mergeid")["int_year"].transform("first")
dat["recent_job_ended_gt_first_int_year"] = (
    dat["most_recent_job_ended"] > dat["first_int_year"]
)

# Reset the index if needed
# dat.reset_index(drop=True, inplace=True)

In [298]:
table(dat["recent_job_ended_gt_first_int_year"])

recent_job_ended_gt_first_int_year
False    4605
True     2218
Name: Count, dtype: int64

In [299]:
dat.loc[
    dat.index[:30],
    [
        "mergeid",
        "first_int_year",
        "int_year",
        "lagged_int_year",
        "working",
        "lagged_working",
        "most_recent_job_started",
        "most_recent_job_ended",
        # "wave_3_response",
        # "wave_7_response"
    ],
]

,mergeid,first_int_year,int_year,lagged_int_year,working,lagged_working,most_recent_job_started,most_recent_job_ended
4,DE-000132-01,2009.0,2009.0,NaN,0.0,NaN,1997.0,2009.0
5,DE-000132-01,2009.0,2011.0,2009.0,1.0,0.0,1997.0,2009.0
6,DE-000132-01,2009.0,2013.0,2011.0,1.0,1.0,1997.0,2009.0
7,DE-000132-01,2009.0,2015.0,2013.0,1.0,1.0,1997.0,2009.0
8,DE-000132-01,2009.0,2017.0,2015.0,0.0,1.0,1997.0,2009.0
11,DE-000554-01,2017.0,2017.0,NaN,1.0,NaN,1982.0,2017.0
12,DE-000554-01,2017.0,2020.0,2017.0,1.0,1.0,1982.0,2017.0
16,DE-000802-01,2013.0,2013.0,NaN,1.0,NaN,1970.0,1971.0
17,DE-000802-01,2013.0,2015.0,2013.0,1.0,1.0,1970.0,1971.0
18,DE-000802-01,2013.0,2017.0,2015.0,0.0,1.0,1970.0,1971.0


In [300]:
dat.sort_values(by=["mergeid", "int_year"], inplace=True)

dat["working_cum"] = dat.groupby("mergeid")["lagged_working"].cumsum()

In [301]:
# # Define boolean masks for the conditions
# condition1 = (dat["lagged_working"] == 1) & (dat["working"] == 1)
# condition2 = dat["lagged_working"].isna()  # == 0
# condition3 = (dat["lagged_working"] == 1) & (dat["working"] == 0)
# condition4 = (dat["lagged_working"] == 0) & (dat["working"] == 0)
# condition5 = (dat["lagged_working"] == 0) & (dat["working"] == 1)
#
# # Calculate 'working_cum_new' based on the conditions
# dat["working_cum_new"] = np.nan  # Initialize to NaN
# dat["working_cum_new"] = np.where(
#     condition2 | condition5,
#     0,  # Set to 0 when lagged_working == 0
#     np.where(
#         condition1 | condition3 | condition4,
#         dat["int_year"] - dat["lagged_int_year"],
#         dat["working_cum_new"],
#     ),
# )
#
# # Calculate the cumulative sum within each 'mergeid' group
# dat["working_cum_new"] = dat.groupby("mergeid")["working_cum_new"].cumsum()


# Define boolean masks for the conditions
lagged_working_none = dat["lagged_working"].isna()  # == 0
lagged_working_false_working_false = (dat["lagged_working"] == 0) & (
    dat["working"] == 0
)
lagged_working_false_working_true = (dat["lagged_working"] == 0) & (dat["working"] == 1)

lagged_working_true_working_true = (dat["lagged_working"] == 1) & (dat["working"] == 1)

# lagged_working == 1 and working == 0
lagged_working_true_working_false = np.where(
    (dat["lagged_working"] == 1)
    & (dat["working"] == 0)
    & (dat["most_recent_job_ended"] < dat["first_int_year"])
    & (dat["first_int_year"] < dat["lagged_int_year"]),  # always the case right?
    np.ceil(
        (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"] / 2,
    ),  # -1 or half point
    # dat["most_recent_job_ended"] - dat["lagged_int_year"]
    #
    np.where(
        (dat["lagged_working"] == 1)
        & (dat["working"] == 0)
        & (dat["lagged_int_year"] < dat["most_recent_job_started"]),
        (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"],
        # else
        (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"],
    ),
)

# condition6 = (
#     (dat["lagged_working"] == 1)
#     & (dat["working"] == 0)
#     & (dat["most_recent_job_ended"] < dat["lagged_int_year"])
# )

# Calculate 'working_cum_new' based on the conditions
dat["working_cum_new"] = np.nan  # Initialize to NaN
dat["working_cum_new"] = np.where(
    lagged_working_none
    | lagged_working_false_working_false
    | lagged_working_false_working_true,
    0,  # Set to 0 when lagged_working == 0
    np.where(
        lagged_working_true_working_true,
        (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"],
        # np.where
        lagged_working_true_working_false,
    ),
)

# Calculate the cumulative sum within each 'mergeid' group
dat["working_cum_new"] = dat.groupby("mergeid")["working_cum_new"].cumsum()

In [302]:
dat["work_exp_cum"] = np.nan

In [303]:
# Sort the DataFrame by mergeid and int_year
dat = dat.sort_values(by=["mergeid", "int_year"])

# Initialize the work_exp_cum column with zeros
dat["work_exp_cum"] = np.nan

# Define the conditions and values for np.select
conditions = dat["lagged_working"] == 1
values = (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"]

# Use np.select to update work_exp_cum
# dat['work_exp_cum'] = np.select(conditions, values, default=0)

dat["work_exp_cum"] = np.where(
    dat["lagged_working"] == 1,
    (dat["int_year"] - dat["lagged_int_year"]) * dat["lagged_exp_weight"],
    0,
)

# Calculate the cumulative sum of work_exp_cum by mergeid
dat["work_exp_cum"] = dat.groupby("mergeid")["work_exp_cum"].cumsum()

In [304]:
def is_weakly_increasing(series):
    return (series.diff().dropna() >= 0).all()


dat_sorted = dat.sort_values(by=["mergeid", "int_year"])

assert (dat["work_exp_cum"] >= 0).all()

# Group the data by mergeid and apply the is_weakly_increasing function
result = dat_sorted.groupby("mergeid")["work_exp_cum"].apply(is_weakly_increasing)

# Assert that all groups have weakly increasing work experience
assert result.all()

In [305]:
# Calculate the cumulative sum of work_exp_cum by mergeid
# dat['work_exp_cum'] = dat.groupby('mergeid')['work_exp_cum'].cumsum()

# Check if work_exp_cum is weakly increasing within each group
dat["work_exp_diff"] = dat.groupby("mergeid")["work_exp_cum"].diff()

# Filter rows where work_exp_cum is not weakly increasing
filtered_dat = dat[dat["work_exp_diff"] < 0]

# Display only the specified columns
# filtered_dat[
#    [
#        "mergeid",
#        "int_year",
#        "working",
#        "lagged_working",
#        "lagged_exp_weight",
#        "work_exp_cum",
#        "work_exp_diff",
#    ]
# ]

In [306]:
dat.loc[
    dat["lagged_exp_weight"] == 1,
    [
        "mergeid",
        "int_year",
        "working",
        "lagged_working",
        "lagged_exp_weight",
        "work_exp_cum",
        "work_exp_diff",
    ],
]

,mergeid,int_year,working,lagged_working,lagged_exp_weight,work_exp_cum,work_exp_diff
6,DE-000132-01,2013.0,1.0,1.0,1.0,2.0,2.0
7,DE-000132-01,2015.0,1.0,1.0,1.0,4.0,2.0
8,DE-000132-01,2017.0,0.0,1.0,1.0,6.0,2.0
17,DE-000802-01,2015.0,1.0,1.0,1.0,2.0,2.0
18,DE-000802-01,2017.0,0.0,1.0,1.0,4.0,2.0
...,...,...,...,...,...,...,...
26424,DE-993972-02,2015.0,1.0,1.0,1.0,2.0,2.0
26425,DE-993972-02,2017.0,0.0,1.0,1.0,4.0,2.0
26466,DE-994888-01,2013.0,0.0,1.0,1.0,2.0,2.0
26468,DE-994888-01,2017.0,0.0,1.0,1.0,4.0,2.0


In [307]:
dat[dat["mergeid"].isin(["DE-003433-01", "DE-004153-02"])][
    [
        "mergeid",
        "int_year",
        "lagged_int_year",
        "working",
        "lagged_working",
        "lagged_exp_weight",
        "work_exp_cum",
        "work_exp_diff",
    ]
]

,mergeid,int_year,lagged_int_year,working,lagged_working,lagged_exp_weight,work_exp_cum,work_exp_diff
134,DE-003433-01,2011.0,NaN,1.0,NaN,NaN,0.0,NaN
135,DE-003433-01,2013.0,2011.0,1.0,1.0,0.5,1.0,1.0
158,DE-004153-02,2007.0,NaN,1.0,NaN,NaN,0.0,NaN
159,DE-004153-02,2009.0,2007.0,0.0,1.0,0.5,1.0,1.0
160,DE-004153-02,2011.0,2009.0,0.0,0.0,0.0,1.0,0.0
161,DE-004153-02,2013.0,2011.0,0.0,0.0,0.0,1.0,0.0
162,DE-004153-02,2015.0,2013.0,0.0,0.0,0.0,1.0,0.0
163,DE-004153-02,2017.0,2015.0,0.0,0.0,0.0,1.0,0.0


In [308]:
np.ceil((2014 - 2011) / 2), (2014 - 2011) / 2

(2.0, 1.5)

In [309]:
dat["working_cum_new"].isna().sum(), dat["working_cum"].isna().sum()

(0, 2774)

In [310]:
table(dat["working_cum_new"])

working_cum_new
0.0    5027
0.5      46
1.0     556
1.5      35
2.0     655
2.5      18
3.0     128
3.5      10
4.0     255
4.5       4
5.0      20
5.5       1
6.0      45
7.0       5
8.0      11
9.0       7
Name: Count, dtype: int64

In [311]:
table(dat["working_cum"])

working_cum
0.0    1851
1.0    1280
2.0     617
3.0     259
4.0      41
5.0       1
Name: Count, dtype: int64

In [312]:
(dat["working_cum"] >= 1).sum(), (dat["working_cum_new"] > 0).sum()

(2198, 1796)

In [313]:
dat.loc[
    dat.index[0:30],
    [
        "mergeid",
        "int_year",
        "lagged_int_year",
        # "first_int_year",
        "most_recent_job_started",
        "most_recent_job_ended",
        # "working",
        "lagged_working",
        "lagged_exp_weight",
        "working_cum",
        "working_cum_new",
        # "work_experience_precise",
        # "work_experience"
        # "sl_re011_1",
        # "sl_re026_1",
        # "sl_re011_2",
        # "sl_re026_2",
        # "sl_re011_3",
        # "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        # "sl_re011_5",
        # "sl_re026_5",
        # "sl_re011_6",
        # "sl_re026_6",
        # "wave_3_response",
        # "wave_7_response"
    ],
].head(24)

,mergeid,int_year,lagged_int_year,most_recent_job_started,most_recent_job_ended,lagged_working,lagged_exp_weight,working_cum,working_cum_new
4,DE-000132-01,2009.0,NaN,1997.0,2009.0,NaN,NaN,NaN,0.0
5,DE-000132-01,2011.0,2009.0,1997.0,2009.0,0.0,0.0,0.0,0.0
6,DE-000132-01,2013.0,2011.0,1997.0,2009.0,1.0,1.0,1.0,2.0
7,DE-000132-01,2015.0,2013.0,1997.0,2009.0,1.0,1.0,2.0,4.0
8,DE-000132-01,2017.0,2015.0,1997.0,2009.0,1.0,1.0,3.0,6.0
11,DE-000554-01,2017.0,NaN,1982.0,2017.0,NaN,NaN,NaN,0.0
12,DE-000554-01,2020.0,2017.0,1982.0,2017.0,1.0,0.0,1.0,0.0
16,DE-000802-01,2013.0,NaN,1970.0,1971.0,NaN,NaN,NaN,0.0
17,DE-000802-01,2015.0,2013.0,1970.0,1971.0,1.0,1.0,1.0,2.0
18,DE-000802-01,2017.0,2015.0,1970.0,1971.0,1.0,1.0,2.0,3.0


In [314]:
# issue
# DE-002173-02

# zero weight because too few hours

# create working_new and lagged_working_new !!

In [315]:
dat.loc[dat["working_cum"] == 0]

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period,freq_visits_mother,freq_visits_father,mother_alive_2,lagged_age_mother,mother_dead,lagged_mother_alive,mother_dead_since_last,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,death_transition,year_mother_death,birth_year_mother,first_mother_death,first_age_mother_death,birth_year_mother_new_2,age_mother_new_2,working,first_int_year,lagged_int_year,lagged_working,full_time,part_time,full_time_nan,most_recent_job_started,most_recent_job_ended,wave_3_response,wave_7_response,both_wave_3_and_7,job_just_started,job_just_ended,exp_weight,lagged_exp_weight,exper_1,exper_2,exper_3,exper_4,exper_5,exper_6,exper_7,exper_8,exper_9,exper_10,exper_11,exper_12,exper_13,exper_14,exper_15,exper_16,exper_17,exper_18,exper_19,exper_20,weight_exper_1,weight_e

In [316]:
# multiply by weight_exper_

# dat["work_experience"] = (
#     dat.filter(like="exper_")
#     .apply(lambda col: col.apply(lambda x: 0 if pd.isna(x) or x < 0 else x), axis=1)
#     .sum(axis=1)
# )


suffixes = range(1, 21)

for i in suffixes:
    dat[f'exper_weighted_{i}'] = dat[f'exper_{i}'] * dat[f'weight_exper_{i}']

In [317]:
table(dat["weight_exper_1"])

weight_exper_1
0.0      91
0.5      31
1.0     500
1.5      18
2.0     500
       ... 
45.0      5
46.0      4
47.0      3
48.0      3
50.0      3
Name: Count, Length: 70, dtype: int64

In [318]:
dat[["mergeid", "int_year", "weight_exper_1"]].head(10)

,mergeid,int_year,weight_exper_1
4,DE-000132-01,2009.0,2.0
5,DE-000132-01,2011.0,2.0
6,DE-000132-01,2013.0,2.0
7,DE-000132-01,2015.0,2.0
8,DE-000132-01,2017.0,2.0
11,DE-000554-01,2017.0,4.0
12,DE-000554-01,2020.0,4.0
16,DE-000802-01,2013.0,1.0
17,DE-000802-01,2015.0,1.0
18,DE-000802-01,2017.0,1.0


In [319]:
# table(dat["exper_1"])
table(dat["work_experience"])

KeyError: 'work_experience'

In [320]:
dat["most_recent_job_exp_weighted"] = dat["corresponding_weight_exper"]

In [321]:
dat["most_recent_job_exp_weighted"]

4        12.0
5        12.0
6        12.0
7        12.0
8        12.0
         ... 
26583     2.0
26585     3.5
26586     3.5
26587     3.5
26588     3.5
Name: most_recent_job_exp_weighted, Length: 6823, dtype: float64

In [322]:
# Define the conditions and corresponding calculations
conditions = [
    dat["most_recent_job_ended"] <= dat["first_int_year"],
    dat["most_recent_job_ended"] > dat["first_int_year"],
]

conditions_new = [
    dat["most_recent_job_ended"] <= dat["first_int_year"],
    #
    # (dat["most_recent_job_ended"] > dat["first_int_year"])
    # & (dat["most_recent_job_started"] > dat["int_year"]),
    # (dat["most_recent_job_ended"] > dat["first_int_year"])
    # & (dat["most_recent_job_ended"] >= dat["int_year"]),
    (dat["most_recent_job_ended"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] >= dat["int_year"]),
    (dat["most_recent_job_ended"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] < dat["int_year"])
    & (dat["int_year"] < dat["most_recent_job_ended"]),
    (dat["most_recent_job_ended"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] < dat["int_year"])
    & (dat["int_year"] >= dat["most_recent_job_ended"]),
    (dat["most_recent_job_ended"] > dat["first_int_year"])
    & (dat["most_recent_job_started"] <= dat["first_int_year"]),
]

calculations = [
    dat["work_experience"] + dat["working_cum"],
    dat["work_experience"]
    + dat["working_cum"]
    - (dat["most_recent_job_ended"] - dat["first_int_year"]),
]

calculations_new = [
    dat["work_experience"] + dat["working_cum_new"],
    dat["work_experience"]
    + dat["working_cum_new"]
    - (dat["most_recent_job_ended"] - dat["most_recent_job_started"]),
    dat["work_experience"]
    + dat["working_cum_new"]
    - (dat["most_recent_job_ended"] - dat["int_year"]),
    dat["work_experience"] + dat["working_cum_new"],
    dat["work_experience"]
    + dat["working_cum_new"]
    - (dat["most_recent_job_ended"] - dat["first_int_year"]),
]

# Create the 'work_experience_precise' variable using np.select
dat["work_experience_precise"] = np.select(conditions, calculations)
dat["work_experience_precise_new"] = np.select(conditions_new, calculations_new)

KeyError: 'work_experience'

In [323]:
dat[dat["mergeid"].isin(["DE-001237-02", "DE-002625-01"])][
    [
        "mergeid",
        "int_year",
        "lagged_int_year",
        "first_int_year",
        # "most_recent_job_started",
        "most_recent_job_ended",
        "working",
        "lagged_working",
        "working_cum",
        "working_cum_new",
        "work_experience_precise",
        "work_experience_precise_new",
        "work_experience",
    ]
]

KeyError: "['work_experience_precise', 'work_experience_precise_new', 'work_experience'] not in index"

In [324]:
dat.loc[
    dat.index[0:30],
    [
        "mergeid",
        "int_year",
        "lagged_int_year",
        # "first_int_year",
        # "most_recent_job_started",
        "most_recent_job_ended",
        "working",
        "lagged_working",
        # "exper_1",
        # "exper_2",
        # "exper_3",
        # "exper_4",
        # "exper_5",
        # "working_cum",
        "working_cum_new",
        "work_experience_precise_new",
        "work_experience",
    ],
]

KeyError: "['work_experience_precise_new', 'work_experience'] not in index"

In [325]:
# DE-001237-02
# correct

# DE-002173-02
# cumulative not correct: 2007 -> 2008
# working --> not working: cum += 1
# most_recent_ended < first_int_year < lagged_int_year

In [326]:
dat.loc[dat["recent_job_ended_gt_first_int_year"]][
    [
        "mergeid",
        "int_year",
        # "lagged_int_year",
        "first_int_year",
        # "most_recent_job_started",
        "most_recent_job_ended",
        "working",
        "lagged_working",
        "working_cum",
        "working_cum_new",
        "work_experience_precise",
        "work_experience_precise_new",
        "work_experience",
        # "sl_re011_1",
        # "sl_re026_1",
        # "sl_re011_2",
        # "sl_re026_2",
        # "sl_re011_3",
        # "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        # "sl_re011_5",
        # "sl_re026_5",
        # "sl_re011_6",
        # "sl_re026_6",
        # "wave_3_response",
        # "wave_7_response"
    ]
]

KeyError: "['work_experience_precise', 'work_experience_precise_new', 'work_experience'] not in index"

In [327]:
dat["work_experience_total"] = dat["work_experience_precise_new"]

KeyError: 'work_experience_precise_new'

In [328]:
table(dat["work_experience_total"])

KeyError: 'work_experience_total'

In [329]:
dat.loc[dat["work_experience_precise_new"] < 0]

KeyError: 'work_experience_precise_new'

In [330]:
# dat.loc[dat["work_experience_total"] > 50][
dat[dat["mergeid"].isin(["DE-360546-01"])][
    [
        "mergeid",
        "int_year",
        # "lagged_int_year",
        "first_int_year",
        "most_recent_job_started",
        "most_recent_job_ended",
        "working",
        "lagged_working",
        "working_cum",
        "working_cum_new",
        "work_experience_precise",
        "work_experience_precise_new",
        "work_experience",
    ]
]

KeyError: "['work_experience_precise', 'work_experience_precise_new', 'work_experience'] not in index"

In [331]:
# dat.loc[dat["work_experience_total"] > 50][
dat[
    dat["mergeid"].isin(
        # ["DE-301352-01", "DE-582036-01", "DE-583077-01", "DE-731816-02", "DE-885680-01"]
        # ["DE-301352-01", "DE-582036-01", "DE-583077-01"]
        ["DE-582036-01"],
    )
][
    [
        "mergeid",
        "int_year",
        # "lagged_int_year",
        "first_int_year",
        # "most_recent_job_started",
        "most_recent_job_ended",
        "working",
        "lagged_working",
        "working_cum",
        "working_cum_new",
        "work_experience_precise",
        "work_experience_precise_new",
        "work_experience",
        # "sl_re011_1",
        # "sl_re026_1",
        # "sl_re011_2",
        # "sl_re026_2",
        # "sl_re011_3",
        # "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        # "sl_re011_5",
        # "sl_re026_5",
        # "sl_re011_6",
        # "sl_re026_6",
        # "wave_3_response",
        # "wave_7_response"
    ]
]

KeyError: "['work_experience_precise', 'work_experience_precise_new', 'work_experience'] not in index"

In [332]:
2017 - 2004

13

In [333]:
dat.loc[
    dat.index[0:30],
    [
        "mergeid",
        "int_year",
        "working",
        "exper_1",
        "exper_2",
        "exper_3",
        "exper_4",
        "exper_5",
        "exper_6",
        "exper_7",
        "exper_8",
        "work_experience",
    ],
]

KeyError: "['work_experience'] not in index"

In [334]:
# dat = dat.drop(dat.filter(like='exper_'), axis=1)

In [335]:
table(dat["work_experience"])

KeyError: 'work_experience'

In [336]:
dat.loc[
    dat.index[:30],
    [
        "mergeid",
        "int_year",
        "age",
        "working",
        "sl_re011_1",
        "sl_re026_1",
        # "sl_re011_2",
        # "sl_re026_2",
        # "sl_re011_3",
        # "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        "sl_re011_5",
        "sl_re026_5",
        # "sl_re011_6",
        # "sl_re026_6",
        "most_recent_job_started",
        "most_recent_job_ended",
        "work_experience",
    ],
]

KeyError: "['work_experience'] not in index"

## Test whether (total) work experience is weakly increasing over int_years 

In [337]:
dat["retro_work_exp"].sum()

109890.5

In [338]:
table(dat["work_exp_cum"])

work_exp_cum
0.0    5027
0.5      46
1.0     555
1.5      36
2.0     654
2.5      18
3.0     126
3.5      10
4.0     258
4.5       4
5.0      20
5.5       1
6.0      45
7.0       5
8.0      11
9.0       7
Name: Count, dtype: int64

In [339]:
dat["work_exp_cum"].sum()

4009.5

In [340]:
dat["work_exp"] = dat["retro_work_exp"] + dat["work_exp_cum"]

In [341]:
table(dat["work_exp"])

work_exp
0.0     1889
0.5        4
1.0      110
1.5        8
2.0      173
        ... 
47.0      14
48.0      18
48.5       2
49.0       5
50.0       9
Name: Count, Length: 96, dtype: int64

In [342]:
dat["work_exp"].sum(), dat["work_exp"].notna().sum()

(113900.0, 6823)

In [343]:
dat.loc[
    dat.index[:30],
    [
        "mergeid",
        "int_year",
        # "age",
        "lagged_working",
        "sl_re011_1",
        "sl_re026_1",
        # "sl_re011_2",
        # "sl_re026_2",
        # "sl_re011_3",
        # "sl_re026_3",
        # "sl_re011_4",
        # "sl_re026_4",
        # "sl_re011_5",
        # "sl_re026_5",
        # "sl_re011_6",
        # "sl_re026_6",
        # "most_recent_job_started",
        "most_recent_job_ended",
        "work_exp_cum",
        "work_experience",
        "work_exp",
    ],
]

KeyError: "['work_experience'] not in index"

In [344]:
def is_weakly_increasing(series):
    return (series.diff().dropna() >= 0).all()


dat_sorted = dat.sort_values(by=["mergeid", "int_year"])

assert (dat["work_exp"] >= 0).all()

# Group the data by mergeid and apply the is_weakly_increasing function
result = dat_sorted.groupby("mergeid")["work_exp"].apply(is_weakly_increasing)

# Assert that all groups have weakly increasing work experience
assert result.all()

In [345]:
dat = dat.sort_values(by=["mergeid", "int_year"])

# Group the data by mergeid and apply the is_weakly_increasing function
result = dat.groupby("mergeid")["work_exp"].apply(is_weakly_increasing)

In [346]:
result.index

Index(['DE-000132-01', 'DE-000554-01', 'DE-000802-01', 'DE-001237-02',
       'DE-001350-01', 'DE-001381-01', 'DE-002106-02', 'DE-002173-02',
       'DE-002625-01', 'DE-002714-01',
       ...
       'DE-995572-01', 'DE-996004-01', 'DE-997333-02', 'DE-997940-02',
       'DE-997975-01', 'DE-998320-03', 'DE-999114-01', 'DE-999375-02',
       'DE-999404-02', 'DE-999990-01'],
      dtype='object', name='mergeid', length=2774)

In [347]:
dat["mergeid"].nunique()

2774

In [348]:
dat[["mergeid", "int_year", "lagged_working", "work_exp"]]

,mergeid,int_year,lagged_working,work_exp
4,DE-000132-01,2009.0,NaN,30.0
5,DE-000132-01,2011.0,0.0,30.0
6,DE-000132-01,2013.0,1.0,32.0
7,DE-000132-01,2015.0,1.0,34.0
8,DE-000132-01,2017.0,1.0,36.0
...,...,...,...,...
26583,DE-999404-02,2017.0,0.0,18.5
26585,DE-999990-01,2013.0,NaN,11.5
26586,DE-999990-01,2015.0,0.0,11.5
26587,DE-999990-01,2017.0,0.0,11.5


In [349]:
mask = ~dat["mergeid"].isin(result.index)

# Use the mask to filter the DataFrame
dat.loc[mask]

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,age_mother,age_father,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,nchild,nchild,wave,years_educ,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,siblings,high_educ,further_educ_max,high_educ_012,high_educ_comb,retired,years_since_retirement,_years_since_retirement,married,married_dn,ever_cared,ever_cared_parents_outside,ever_cared_parents_within,ever_cared_parents,care_in_year,care_experience_,care_experience,light_care,intensive_care,mother_alive,health_mother,father_alive,health_father_3,dist_father,dist_mother,parents_live_close,worked_last_period,freq_visits_mother,freq_visits_father,mother_alive_2,lagged_age_mother,mother_dead,lagged_mother_alive,mother_dead_since_last,age_mother_first,int_year_mother_first,age_year_mother_new,age_mother_death,death_transition,year_mother_death,birth_year_mother,first_mother_death,first_age_mother_death,birth_year_mother_new_2,age_mother_new_2,working,first_int_year,lagged_int_year,lagged_working,full_time,part_time,full_time_nan,most_recent_job_started,most_recent_job_ended,wave_3_response,wave_7_response,both_wave_3_and_7,job_just_started,job_just_ended,exp_weight,lagged_exp_weight,exper_1,exper_2,exper_3,exper_4,exper_5,exper_6,exper_7,exper_8,exper_9,exper_10,exper_11,exper_12,exper_13,exper_14,exper_15,exper_16,exper_17,exper_18,exper_19,exper_20,weight_exper_1,weight_e

In [350]:
mask

4        False
5        False
6        False
7        False
8        False
         ...  
26583    False
26585    False
26586    False
26587    False
26588    False
Name: mergeid, Length: 6823, dtype: bool

In [ ]:
# Assert that all groups have weakly increasing work experience
assert (
    result.all()
)  # , "work_experience_precise_new is not weakly increasing for some mergeids"

In [ ]:
def test_work_experience_non_negative():
    """Checks if all elements are non-negative"""
    assert (dat["work_experience_precise_new"] >= 0).all()